---
title: "Caractérisation et évolution des précipitations extrêmes horaires en France à partir d’un modèle régional de climat à convection profonde résolue"

lang: fr
bibliography: bibliography.bib

format:
  pdf:
    documentclass: article
    papersize: a4
    fontsize: 11pt
    geometry:
      - margin=1cm
    number-sections: true
    toc: false

execute:
  eval: true
  echo: false
  warning: false
  error: false
---

*Les cartes étant rasterisées, il est possible d’appliquer un zoom à n’importe quel niveau afin de mieux distinguer les détails visuels. Toutefois, comme il s’agit de données en pixels et non vectorielles, un agrandissement trop important entraîne une perte de netteté et l’apparition de pixellisation.*

\small Repository : https://github.com/NCSdecoopman/ExtremePrecipit \
\small Application : https://huggingface.co/spaces/ncsdecoopman/ExtremePrecipit

In [ ]:
from __future__ import annotations

import sys
import os
from pathlib import Path
from typing import Tuple

import pandas as pd

from svgutils.transform import fromfile, SVGFigure
import cairosvg

def _to_px(value: str | None) -> float:
    """
    Convertit une longueur SVG (px, mm, cm, pt, in) en pixels (float).

    Si `value` est None ou vide, retourne 0.
    """
    if not value:
        return 0.0
    value = value.strip()
    num = ''
    unit = ''
    for ch in value:
        if ch.isdigit() or ch in '.-':
            num += ch
        else:
            unit += ch
    if not num:
        return 0.0
    numf = float(num)
    unit = unit.strip().lower()
    if unit in ('', 'px'):
        return numf
    if unit == 'mm':
        return numf * 3.779527559055  # 96 dpi
    if unit == 'cm':
        return numf * 37.79527559055
    if unit == 'in':
        return numf * 96
    if unit == 'pt':
        return numf * 1.3333333333333  # 1 pt = 1/72 in
    # Fallback: assume pixels
    return numf


def _dims(fig) -> Tuple[float, float]:
    """
    Renvoie (width, height) de `fig` en pixels.

    1) viewBox (les 2 derniers termes)
    2) attributs width/height de la racine
    3) fig.get_size()
    """
    root = fig.root  # Correction ici : on accède à la balise <svg>
    viewbox = root.get('viewBox')
    if viewbox:
        parts = [p for p in viewbox.replace(',', ' ').split() if p]
        if len(parts) == 4:
            return float(parts[2]), float(parts[3])

    # attributs width/height sur la balise <svg>
    w_attr = root.get('width')
    h_attr = root.get('height')
    if w_attr and h_attr:
        return _to_px(w_attr), _to_px(h_attr)

    # fallback
    w, h = fig.get_size()
    return _to_px(w), _to_px(h)


def assemble(arome: Path, stations: Path, legend: Path, output: Path) -> None:
    # Charger les 3 SVG
    fig_arome = fromfile(str(arome))
    fig_stations = fromfile(str(stations))
    fig_legend = fromfile(str(legend))

    # Dimensions
    w_arome, h_arome = _dims(fig_arome)
    w_stations, h_stations = _dims(fig_stations)
    w_leg, h_leg = _dims(fig_legend)

    # Canvas global
    w_maps = max(w_arome, w_stations)
    h_maps = h_arome + h_stations
    height = h_maps  # somme des deux cartes

    # Facteur d'échelle pour que la légende fasse 1.5 fois la hauteur de la carte AROME
    scale_leg = (1.5 * h_arome) / h_leg
    w_leg_scaled = w_leg * scale_leg
    h_leg_scaled = h_leg * scale_leg  # = 1.5 * h_arome

    width = w_maps + w_leg_scaled  # légende à droite

    canvas = SVGFigure(f"{width}px", f"{height}px")
    canvas.root.set('viewBox', f"0 0 {width} {height}")

    # Racines
    root_arome = fig_arome.getroot()
    root_stations = fig_stations.getroot()
    root_legend = fig_legend.getroot()

    # Redimensionnement de la légende
    root_legend.scale(scale_leg, scale_leg)

    # Positionnement
    root_stations.moveto(0, h_arome)
    # Centrage vertical de la légende sur la hauteur totale
    y_leg = (h_maps - h_leg_scaled) / 2
    root_legend.moveto(w_maps, y_leg)

    canvas.append([root_arome, root_stations, root_legend])

    os.makedirs(os.path.dirname(str(output)), exist_ok=True)
    canvas.save(str(output))
    pdf_path = str(output)[:-4] + ".pdf"
    cairosvg.svg2pdf(url=str(output), write_to=pdf_path)
    return str(output)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

def combined_metrics_df(name_file: str):
    base_dir = Path("../outputs")         # dossier racine à explorer
    csv_paths = list(base_dir.rglob(name_file))

    frames = []
    for path in csv_paths:
        try:
            df = pd.read_csv(path)
            df["source"] = str(path.relative_to(base_dir))  # garder la provenance
            frames.append(df)
        except Exception as exc:                             # CSV illisible
            print(f"⚠️  Fichier ignoré {path}: {exc}")

    if not frames:
        raise SystemExit("Aucun metrics.csv trouvé !")

    # Concaténation puis export
    combined = pd.concat(frames, ignore_index=True)

    # --- harmonisation des échelles ---
    # cas quotidien_reduce
    mask_q = combined["source"].str.contains("quotidien_reduce", na=False)
    combined.loc[mask_q & (combined["echelle"] == "quotidien"), "echelle"] = "quotidien_reduce"

    # cas horaire_reduce
    mask_h = combined["source"].str.contains("horaire_reduce", na=False)
    combined.loc[mask_h & (combined["echelle"] == "horaire"), "echelle"] = "horaire_reduce"

    return combined

combined = combined_metrics_df("metrics.csv")
combined_signif = combined_metrics_df("metrics_signif.csv")
combined_signif["col_calculate"] = combined_signif["col_calculate"].astype(str) + "_signif"
combined_all = pd.concat([combined, combined_signif], ignore_index=True)


import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker


def plot_r_by_season(
    df,
    echelles,
    col_values,          # <-- plusieurs indicateurs (ex. ['numday', 'mean', 'mean-max'])
    nom_fichier,         # préfixe du fichier (le numéro du facet est ajouté automatiquement)
    col_plot: str="r"
):
    """
    Trace un bar‑chart groupé « r » par saison, pour plusieurs indicateurs.
    Produit un seul PNG contenant un facet‑grid (nrows × ncols).

    Parameters
    ----------
    df : pandas.DataFrame
    echelles : list[str]
        Sous-ensemble d'échelles à garder (ex. ['quotidien', 'horaire'])
    col_values : list[str]
        Liste des valeurs cibles de la colonne 'col_calculate'
    nom_fichier : str
        Nom de base du fichier PNG (sans extension)
    """
    ncols=len(col_values)

    label_map = {
        "numday":  "Nombre de jour de pluie",
        "mean":    "Cumul de précipitations",
        "mean-max": "Moyenne des maxima",
        "z_T_p":   "Tendance significative et non significative",
        "z_T_p_signif":   "Tendance significative"
    }

    # ---------------------------------------------
    # 0) Préparation de la grille de sous‑graphes
    # ---------------------------------------------
    n_facets = len(col_values)
    ncols = max(1, min(ncols, n_facets))
    nrows = math.ceil(n_facets / ncols)

    fig, axes = plt.subplots(
        nrows=nrows,
        ncols=ncols,
        figsize=(12, 5 * nrows),
        sharey=True
    )
    # axes → tableau 1D pour itération uniforme
    axes = np.atleast_1d(axes).flatten()

    # Pour récupérer une seule fois handles/labels de la légende
    first_handles, first_labels = None, None

    # ------------------------------------------------------------------
    # Boucle sur chaque col_value demandé → un subplot par indicateur
    # ------------------------------------------------------------------
    for ax, col_value in zip(axes, col_values):
        # --- 1) Filtrage identique ---
        sub = df.loc[
            (df["echelle"].isin(echelles)) &
            (df["col_calculate"] == col_value),
            ["season", "echelle", col_plot]
        ]
        if sub.empty:
            ax.set_visible(False)
            ax.set_title(f"Aucune donnée pour '{col_value}'")
            continue

        sub["season"] = sub["season"].str.upper()

        # --- 2) Ordre fixe ---
        ordre_total = [
            "HYDRO", 
            "SON", "DJF", "MAM", "JJA",
            "OND", "JFM", "AMJ", "JAS",
            "SEP", "OCT", "NOV", "DEC", "JAN", "FEV", 
            "MAR", "AVR", "MAI", "JUI", "JUILL", "AOU"
        ]
        saisons_pres = [s for s in ordre_total if s in sub["season"].unique()]

        # --- 3) Renommage des échelles ---
        rename_map = {
            "quotidien":        "Journalière (1959‑2022)",
            "quotidien_reduce": "Journalière (1990‑2022)",
            "horaire":          "Horaire (1990‑2022)",
        }

        # --- 4) Pivot ---
        pivot = (
            sub.pivot(index="season", columns="echelle", values=col_plot)
               .rename(columns=rename_map)
               .reindex(saisons_pres)
        )

        # --- 5) Colonnes dans l’ordre demandé ---
        ordre_cols = [
            rename_map[k] for k in ["quotidien", "quotidien_reduce", "horaire"]
            if rename_map[k] in pivot.columns
        ]
        pivot = pivot[ordre_cols]

        # --- 6) Couleurs ---
        couleurs = ["black", "dimgray", "lightgray"][:pivot.shape[1]]

        # --- 7) Bar chart groupé ---
        x = np.arange(len(pivot))
        largeur = 0.8 / pivot.shape[1]

        for i, (col, couleur) in enumerate(zip(pivot.columns, couleurs)):

            bars = ax.bar(
                x + i * largeur,
                pivot[col],
                width=largeur,
                color=couleur,
                label=col
            )

        # --- 8) Axes, grille, ticks ---
        ax.set_xticks(
            x + largeur * (pivot.shape[1] - 1) / 2,
            pivot.index
        )
        ax.tick_params(axis='x', rotation=45)   # Rotation ici
        ax.set_xlabel("")
        if col_plot=="me":
          ax.set_ylabel("Δ (AROME - Stations) (mm/h)")
        else:
          ax.set_ylabel(col_plot)
        ax.set_title(label_map.get(col_value, col_value))   # titre lisible pour chaque subplot)

        if col_plot=="r":
          ax.set_ylim(top=1)   # <-- Forcer ymax à 1
          ax.yaxis.set_major_locator(mticker.MultipleLocator(0.1))
          ax.yaxis.set_minor_locator(mticker.MultipleLocator(0.02))
        else:
          ax.yaxis.set_major_locator(mticker.MultipleLocator(0.5))
          ax.yaxis.set_minor_locator(mticker.MultipleLocator(0.25))

        ax.grid(axis="y", which="major", linestyle="--", alpha=0.8)
        ax.grid(axis="y", which="minor", linestyle=":",  alpha=0.5)

        # mémoriser les handles/labels la première fois
        if first_handles is None:
            first_handles, first_labels = ax.get_legend_handles_labels()

        # supprimer la légende locale seulement si elle existe
        leg = ax.get_legend()
        if leg is not None:
            leg.remove()


    # ------------------------------------------------------------------
    # 9) Légende globale (ligne en‑haut, centrée)
    # ------------------------------------------------------------------
    if first_handles:
        fig.legend(
            first_handles,
            first_labels,
            loc="lower center",
            ncol=len(first_labels),
            bbox_to_anchor=(0.5, 1.02)
        )

    fig.tight_layout()
    fig.subplots_adjust(top=0.90)  # laisser de l’espace pour la légende globale

    # ------------------------------------------------------------------
    # 10) Sauvegarde
    # ------------------------------------------------------------------
    fig.savefig(f"figures/{nom_fichier}.png", dpi=500, bbox_inches="tight")
    plt.close(fig)

## Introduction et contexte

### L'effet du réchauffement climatique sur les précipitations

Le changement climatique entraine un réchauffement de l’air à la surface de la planète, plus marqué sur les continents que sur les océans [@IPCC2021]. L'augmentation est de +1,1°C à l'échelle mondiale, +1,7°C à l'échelle de la France métropolitaine et +2°C à l'échelle des Alpes françaises par rapport à l’ère préindustrielle. Par ailleurs, la relation de Clausius-Clapeyron montre que l'air chaud peut contenir plus d'humidité (+7%/°C) [@clapeyron1834]. Autrement dit, à +1°C, une même masse d’air pourra contenir 7% de vapeur d’eau en plus. Du fait de la poussée d'Archimède, l'air chaud entourée d'air plus froid aura tendance à monter. L'ascension de l'air chaud dans l'atmosphère entraîne son refroidissement adiabatique, provoquant la condensation de la vapeur d'eau qui se transforme en précipitations [@meteofrance]. Toutefois, en conditions calmes, la partie centrale de la distribution des précipitations ne parvient pas à tirer pleinement parti de cet excédent d’humidité. Il existe des contraintes énergétiques (bilan radiatif, évaporation, échanges océan‑air) et dynamiques (subsidence, vent synoptique) qui limitent l’augmentation des précipitations moyennes à seulement 1–3%/°C [@IPCC2021]. En revanche, lors d’événements convectifs intenses (orage, cyclogenèse rapide) l’ascension rapide condense presque intégralement ce surplus, et les pluies maximales sur des durées courtes augmentent de 5 à 8% par °C, soit presque au même rythme que le potentiel théorique. Les extrêmes de précipitation suivent de près la loi de Clausius–Clapeyron, tandis que la pluie moyenne reste sous l’influence de nombreux autres facteurs énergétiques et dynamiques [@ogorman2015contrasting]. Ainsi, en réponse au réchauffement climatique il existe une augmentation *théorique* des précipitations extrêmes. Cette augmentation est néanmoins variable suivant les changements de circulations atmosphériques et peut être amplifiée localement [@blanchet2021explaining].

### Les précipitations extrêmes

Les précipitations extrêmes sont définies comme des épisodes de fortes pluies intenses sur de courtes durées (1h à 24h) et correspondent à la queue de la distribution des intensités pluviométriques. Il n'existe pas de définition consensuelle de ce qui caractérise un *extrême*. Certains auteurs étudient des précipitations d'intensités au-dessus du 99e percentile ou des maxima saisonniers/annuels. D'autres, considèrent les précipitations extrêmes comme des événements qui sont rarement ou jamais rencontrés dans la vie d'un humain (par exemple, des niveaux de précipitations aux quels on s'attend une fois tous les 10, 20 ou 50 ans). Les extrêmes sont au coeur des préoccupations climatiques et sociétales. Ces épisodent provoquent 90% des coûts liés aux inondations, aux glissements de terrain et aux ruptures d’infrastructures [@IPCC_2022_WGIII]. En 2024, de nombreux événements de ce type ont fait l'actualité, par exemple au Népal ou en Afghanistan et en Europe centrale avec l'Espagne orientale et la France [@WMO2025]. En particulier, en juin 2024, des pluies intenses en haute altitude qui ont contribué à des inondations majeures dans le massif des Écrins [@Blanc2024] ; en octobre 2024, plus de 600 mm en 48h dans le département de l'Ardèche [@MeteoFrance2024_episodesArdeches] ; en mai 2025, des orages courts mais extrêmement intenses (localement dépassant 120 mm/h) ont causé des dommages généralisés dans le sud du département du Var [@MeteoFrance2025].

#### A l'échelle journalière

Pour les extrêmes journaliers, le signal moyen à l’échelle globale est orienté à la hausse : la fréquence et l’intensité augmentent depuis les années 1950, avec des hausses significatives rapportées dans une majorité de régions et de mailles terrestres. Par exemple, le nombre annuel de jours dépassant 10 mm/j de précipitation a augmenté de +0,2 à +0,3 jour par décennie sur 1951–2010 [@Donat2013], avec seulement 29% des mailles terrestres présentant une hausse statistiquement significative. Les événements de type pluie extrême journalière avec retour décennal ont progressé d’environ +6,7 % dans 71 % des régions terrestres [@IPCC2021]. En France, les signaux sont beaucoup plus hétérogènes et souvent moins nets que pour la température, avec de fortes variations régionales. Dans la plupart des régions, les tendances restent faibles ou non significatives, et ce n’est que dans certaines zones, notamment le sud-est, que l’on détecte des signaux clairs. Dans cette région, la fréquence des épisodes méditerranéens extrêmes (cumuls > 200 mm en 24h) a doublé entre 1961 et 2020, même si la variabilité interannuelle reste très marquée [@meteofrance2024_episodesMediterraneens]. L’intensité moyenne des extrêmes journaliers y a augmenté de +22% entre 1961 et 2015 [@Ribes2019]. À l’échelle nationale, aucune tendance significative n’était détectable sur les maxima annuels de précipitations quotidiennes jusqu’aux années 1990. Depuis, certaines études rapportent des augmentations de +20 % à +40 %, mais ces estimations restent sensibles à la période et à la méthode d’analyse. Dans le sud-est des Alpes, la hausse du niveau de précipitation journalière attendue en moyenne tous les 20 ans en automne entre 1958 et 2017 pourrait atteindre un ordre de grandeur comparable à sa valeur moyenne (environ +100%) [@blanchet2021explaining]. Enfin, les projections montrent qu’un scénario de réchauffement de +4°C pourrait conduire à une hausse moyenne d’environ +15% des pluies extrêmes journalières en France et jusqu’à +20 % dans la moitié nord [@soubeyroux:hal-04991790]. Il existe une forte variabilité spatiale et interannuelle.

#### A l'échelle horaire

Les extrêmes horaires, mesurés sur des pas de temps de 1h ou moins, sont essentiels pour caractériser les phénomènes convectifs intenses (averses orageuses, orages stationnaires) souvent responsables de crues éclaires. Faute de séries longues et spatialisées, il n’existe pas d’analyse systématique des tendances sub‑journalières à l’échelle globale ; les données sont souvent éparses, courtes et non significatives. Malgré cela, plusieurs études régionales détectent une intensification des extrêmes horaires sur presque tous les continents, mais la confiance dans une hausse globale reste très faible [@IPCC2021]. Un accroissement des pluies extrêmes est relevée aux États-Unis, en Chine (l'été), en Australie (sur l'année), en Afrique du Sud (l'été), en Inde, en Malaisie, et en Italie. Plusieurs travaux mettent en évidence, selon la méthode et la région considérées, des sensibilités de l’ordre de +7% à +13% par °C, c’est‑à‑dire jusqu’à deux fois la pente de Clausius‑Clapeyron [@molnar2015relation]. En France, peu d'études régionales caractérisent explicitement cette hausse des extrêmes. Les valeurs maximales observées sur 1h atteignent maintenant 40–60 mm dans les événements méditerranéens majeurs, contre 30–40 mm dans les années 1980–1990 [@meteofrance2024_episodesMediterraneens]. Seul l'étude de @Berghald2025 semble chiffrer l'évolution des tendances des extrêmes horaires dans les Alpes. Cependant, la tendance sur le niveau de retour horaire est faible à non significative, sans cohérence spatiale ni saisonnière marquée, en contraste avec les résultats clairs observés au pas de temps journalier. Cela montre que les extrêmes horaires ne montrent pas (encore) de signal climatique clair [@Soubeyroux01022015].

### Les données disponibles

#### Relevés Météo-France

Le réseau climatologique s'est développé à la fin du XIXᵉ siècle. Les lectures étaient manuelles, généralement une fois par jour (autour de 6h UTC, ce qui définit encore souvent le jour météorologique). Un réseau à l’échelle nationale est disponible à partir des années 1950–1960, période où la couverture devient suffisamment dense. Des centaines de séries précipitations existent aujourd’hui [@meteo-france_2020_breve_observation_classique]. A l'échelle horaire, les premières stations automatiques opérationnelles (SATIN) datent de 1967. La généralisation et la modernisation du réseau temps réel se fait surtout avec RADOME à partir de 1996, ce qui rend les cumuls horaires réellement disponibles et diffusés à grande échelle à partir des années 1990–2000. Aujourd’hui la mosaïque radar nationale fournit des données toutes les 5 min à 1 km, accessibles en archives publiques. Les lames d’eau ANTILOPE (radar + pluviomètres) sont produites opérationnellement, avec un pas de 15 min disponible depuis 2017.

#### Apport des modèles climatiques

La simulation des précipitations par modèle climatique apporte 1) des données antérieures aux données observées de 1990 ; et 2) des champs spatiaux complets et cohérents physiquement utile en zones peu instrumentées. Les modélisations GCM (General Circulation Model) ne se sont pas recommandées pour cette étude : la convection est paramétrée, les précipitations horaires sont souvent trop lissées et mal positionnées avec une résolution 12–200 km. L’émergence des modèles climatiques régionaux à convection explicite (CPMs, Convection-Permitting Model) ouvre une opportunité inédite : ils permettent de simuler, avec réalisme, la dynamique des précipitations intenses à fine échelle spatiale et temporelle sur de longues périodes. Pour cette étude le modèle doit être RCM (Regional Climate Model, 1–4 km) à convection explicite (CPM) pour représenter explicitement l’initiation, l’organisation et la propagation des systèmes convectifs et leurs extrêmes horaires. Les modèles répondant à ces critères sont : WRF 3 km, COSMO‑CLM 2.8 km, AROME‑Climate 2.5 km [@cnrm_arome2007], ICON-LEM 2.5 km, UKCP CPM 2.2 km, CP4‑Africa, ensembles EURO-CP / FPS‑Convection.

#### Apport des réanalyses

Un CP-RCM doit être initialisé (champs 3D de température, humidité, pression, vent, etc.) et forcé aux limites latérales. Historiquement, ces forçages utilisaient ERA-Interim (80 km / 6h / 1979-2019), désormais remplacé par ERA5 (25–31 km / 1h / 1949-présent), qui permet d’alimenter dynamiquement un modèle régional à maille fine. ERA5 étant une réanalyse, le modèle régional qu’elle force vise à reconstituer les conditions météorologiques passées heure par heure, et non à produire une simulation climatique prospective ou stochastique. Cette approche permet par exemple de caractériser les environnements convectifs [@ecmwf_era5_data_documentation], puis d'affiner avec un CPM pour simuler explicitement la convection. Le produit AROME (Application of Research to Operations at MEsoscale, 2,5 km / 1h) forcé par ERA5 sur la période 1959-2022 répond à ces critères et a été retenu comme modèle numérique pour cette étude ; à notre connaissance, il n’a encore jamais été évalué.

### Intérêt de l’étude dans le paysage scientifique

Si les tendances des extrêmes journaliers sont désormais bien documentées à l’échelle mondiale et régionale, les précipitations horaires — pourtant cruciales pour les inondations soudaines — souffrent encore d’un manque de documentation, notamment en France. Les observations longues et homogènes y sont rares, les tendances peu significatives, et les études ciblées encore peu nombreuses. Ce déficit de connaissance s'explique à la fois par la faible durée des séries horaires disponibles et par la complexité physique des processus convectifs à l’origine de ces extrêmes. La réanalyse AROME forcée par ERA5 (1959–2022) fournit un jeu de données unique pour étudier les précipitations horaires extrêmes en France. Toutefois, la validité des extrêmes simulés par ce modèle n'a jamais été évalué, tant du point de vue statistique que physique. Notre étude s’inscrit dans cette lacune. Elle vise à **évaluer finement la capacité du modèle AROME à reproduire les extrêmes horaires observés**, en mettant en regard les tendances modélisées et observées sur plusieurs décennies. Elle apporte ainsi une double contribution : 1) méthodologique, en évaluant de nouvelles simulations au regard d'observations d’extrêmes ; 2) climatique, en documentant pour la première fois les **évolutions temporelles** des extrêmes horaires en France sur 60 ans à partir d’un modèle à convection explicite validé. Ce travail éclaire ainsi la manière dont les extrêmes horaires évoluent dans un contexte de réchauffement climatique, tout en évaluant la pertinence des modèles climatiques CPM pour les études d’impact hydrométéorologique à l’échelle locale.

## Méthodologie

L'étude se concentre sur la France métropolitaine. Dans un premier temps, l’objectif est de valider le modèle (points de grille AROME) vis‑à‑vis des observations (stations Météo‑France) à l’aide d’indicateurs simples (corrélation et biais) mais représentatifs du régime de précipitations. Cette étape est conduite en contexte stationnaire, c’est‑à‑dire sans chercher encore à interpréter des tendances à long terme ou des ruptures de régime. Une fois cette validation stationnaire établie, l’analyse sera étendue à des contextes non stationnaires (tendances et variabilité des extrêmes).

### Données utilisées

#### Données observées

Ce travail utilise les données de précipitations issues d'observations Météo-France [@meteofrance2024] au pas de temps journalier (1959-2022) et horaire (1990-2022) représentées sur la figure 1. Depuis les années 2000 des procédures d’homogénéisation (PRODIGE, puis HOMER) ont été mises en place dans la BDClim [@meteo-france_2020_breve_observation_classique].

\setlength{\tabcolsep}{0pt}  

| \small Stations au pas de temps journalier (1959-2022) | \small Stations au pas de temps horaire (1990-2022) |
| :---: | :---: |
| ![](../outputs/hist/dispo/quotidien/hydro/hist_len_years.pdf) | ![](../outputs/hist/dispo/horaire/hydro/hist_len_years.pdf) |
| n = 8198 | n = 2315 |


\captionof{figure}{\small Distribution du nombre d'années hydrologiques comportant au plus 10\% de valeurs manquantes des stations Météo-France au pas de temps journalier (1959–2022) et horaire (1990–2022).}

#### Données modélisées

Le modèle numérique CP-RCM AROME, tout récemment forcé par réanalyse ERA5 offre, parmi différentes variables de sortie, des données de précipitations horaires de 1959 à 2022 à une résolution de 2,5 km sur le champ de calcul de la figure 2 [@arome2014]. En France métropolitaine 87 536 points sont générés. Dans le cadre d'un second stage auprès de Cécile Caillaud de Météo-France, Mathis Chevé a réalisé une étude conjointe sur la relation de Clausius-Clapeyron montrant que les tendances de températures du modèle sont deux fois plus faibles que les tendances observées. Il est donc à noter pour la suite que la part des tendances des extrêmes liée à Clausius-Clapeyron sera théoriquement deux fois plus faible que les tendances observées.

\
\begin{center}
  \centering \includegraphics[width=0.5\textwidth]{fig/espace_definition.pdf}
\end{center}
\
\captionof{figure}{\small Cartographie du champ de calcul du modèle numérique AROME.}

#### Filtrage des séries temporelles

En chaque point géographique on calcule la part de données manquantes dans la série temporelle pour chaque saison (ou mois) et année. On élimine les années dont cette part dépasse le seuil fixé de 10%. On déduit le nombre d’années valides restantes. On ne garde que les points géographiques qui possèdent au moins le nombre minimal d’années exigé (50 ans pour la période 1959-2022 et 25 ans pour la période 1990-2022). Les analyses suivantes ne portent plus que sur ce sous‑ensemble de stations et d’années.

### Définition des saisons

Les saisons sont définies par **SON** pour septembre (**SEP**) octobre (**OCT**) novembre (**NOV**), **DJF** pour décembre (**DEC**) janvier (**JAN**) février (**FEV**), **MAM** pour mars (**MAR**) avril (**AVR**) mai (**MAI**), et **JJA** pour juin (**JUI**) juillet (**JUILL**) août (**AOU**). L'année hydrologique (**HYDRO**) est définie comme la période allant du 1er septembre de l'année N au 31 août de l'année N+1.

### Calcul des indicateurs descriptifs

À partir des données obtenues de Météo-France et d'AROME, le nombre de jour de pluie (seuil fixé à 1 mm/j), le cumul et les maxima de précipitations journalières et horaires ont été calculées pour chaque point de grille AROME ou station, année et mois/saison sur les période 1959-2022 et 1990-2022.

### Vers une généralisation par modélisation statistique

Les statistiques descriptives ne permettent pas d’extrapoler, quantifier l’incertitude et la significativité et de relier proprement les tendances observées aux mécanismes physiques. On ne peut donc pas transformer des constats descriptifs en résultats exploitables et comparables. Pour répondre à ces questions, nous mobilisons la théorie des valeurs extrêmes et en particulier le théorème de Fisher–Tippett–Gnedenko [@coles2001introduction]. Les maxima, une fois normalisés, convergent vers une famille généralisée des valeurs extrêmes (GEV - Generalized Extreme Value). C’est l’analogue, pour les maxima, du théorème central limite pour les sommes. Ce théorème stipule que, si $(M_n)$ désigne la suite des maxima normalisés d’échantillons indépendants et identiquement distribués, alors sous des conditions générales, la loi limite de $M_n$ appartient à la famille généralisée des valeurs extrêmes. La GEV unifie trois familles classiques : Gumbel ($\xi \to 0$), Fréchet ($\xi > 0$) et Weibull ($\xi < 0$). La *bonne* famille étant déterminée automatiquement par l’estimation de $\xi$ qui contrôle la lourdeur de la queue. Nous ajustons des lois GEV aux maxima, d’abord dans un cadre stationnaire pour valider le bon comportement du modèle, puis dans un cadre non‑stationnaire où les paramètres de la GEV dépendent du temps. Cette étape nous permet de comparer de manière cohérente les niveaux de retour entre observations et AROME, d’estimer leurs tendances et de relier ces tendances aux mécanismes physiques.

#### Définitions

Si on note $x$ une réalisation de la variable aléatoire $X$, représentant le maximum annuel de précipitations en un point spatial donné, alors la loi GEV est une loi de probabilité continue paramétrée par le triplet $\theta = (\mu, \sigma, \xi)$ — respectivement la position, l’échelle (strictement positive) et la forme dont la fonction de répartition cumulative est :

$$
F(x;\mu ,\sigma ,\xi) = \exp\left\{-\left[1 + \xi\frac{x - \mu}{\sigma}\right]^{-1/\xi}\right\}, \quad 1 + \xi\frac{x - \mu}{\sigma} > 0
$$

#### Covariable temporelle

On dispose d'une série temporelle de $n$ maxima annuels de précipitations pour un point géographique. On suppose ces maxima indépendants. Ces observations sont notées $\{x_1, x_2, \dots, x_n\}$ où chaque $x_i$ est un maximum annuel de précipitations observé à l'année $t_i$ et qui suit une loi GEV dont les paramètres dépendent, analytiquement, de l'année $i$. On transforme l’année $t_i$ en une covariable normalisée notée $\tilde{t}_i$. Cette transformation est simplement réalisée pour des raisons numériques mais elle ne change rien au résultat théorique. On crée également une covariable temporelle avec point de rupture noté $t_+$ tel que :

$$
\tilde{t}_i = \frac{t_i - t_{\min}}{t_{\max} - t_{\min}} \quad \text{avec} \quad \begin{cases}
t_{min} = \min_i t_i \\
t_{max} = \max_i t_i
\end{cases} \quad \text{et} \quad
\tilde{t}_{i}^\ast =
\begin{cases}
0 & \text{si } t_i < t_+ \\
\displaystyle \frac{t_i - t_+}{t_{\max} - t_+} & \text{si } t_i \ge t_+
\end{cases}
$$

Ce codage permet d’appliquer une pente temporelle seulement après la date de rupture, avec une covariable encore normalisée sur $[0,1]$ dans la portion post-rupture.

#### Modèles utilisés

:::{.callout}
Soit la covariable $t \in \mathbb{N} \mid t_{\min} \leq t \leq t_{\max}$. Le modèle stationnaire est défini par $M_0(\theta_0)$ et $\theta_0 = (\mu_0, \sigma_0, \xi_0)$ avec $\mu(t) = \mu_0$ ; $\sigma(t) = \sigma_0$ ; $\xi(t) = \xi_0$. Les modèles non stationnaires sont définis par :

$$
\begin{array}{c@{\qquad}c@{\qquad}c}
% -------- Colonne 1 --------
\begin{array}{c}
\begin{aligned}
M_1(\theta_1)\\[-0.2ex]
\theta_1 = (\mu_0, \mu_1, \sigma_0, \xi_0)
\end{aligned}\\[0.4ex]
\begin{cases}
\mu(t) = \mu_0 + \mu_1\, t \\
\sigma(t) = \sigma_0 \\
\xi(t) = \xi_0
\end{cases}
\end{array}
&
% -------- Colonne 2 --------
\begin{array}{c}
\begin{aligned}
M_2(\theta_2)\\[-0.2ex]
\theta_2 = (\mu_0, \sigma_0, \sigma_1, \xi_0)
\end{aligned}\\[0.4ex]
\begin{cases}
\mu(t) = \mu_0 \\
\sigma(t) = \sigma_0 + \sigma_1\, t \\
\xi(t) = \xi_0
\end{cases}
\end{array}
&
% -------- Colonne 3 --------
\begin{array}{c}
\begin{aligned}
M_3(\theta_3)\\[-0.2ex]
\theta_3 = (\mu_0, \mu_1, \sigma_0, \sigma_1, \xi_0)
\end{aligned}\\[0.4ex]
\begin{cases}
\mu(t) = \mu_0 + \mu_1\, t \\
\sigma(t) = \sigma_0 + \sigma_1\, t \\
\xi(t) = \xi_0
\end{cases}
\end{array}
\end{array}
$$

:::

Lorsqu'un point de rupture noté $t_+$ est introduit, on note : 

$$
t^\ast = t \cdot \mathbb{1}_{t > t_+} \quad \text{avec} \quad t_+ \in \mathbb{N}
$$

Les modèles $M_1$, $M_2$ et $M_3$ deviennent respectivements $M_1^\ast$, $M_2^\ast$ et $M_3^\ast$. Sur cette même notation $\theta_i$ devient $\theta^\ast_i$ avec $i \in \{1, 2, 3\}$. Dans cette étude, on réalise les modélisations stationnaire et non-stationnaires avec pour covariable l'année et un effet temporel sur $\mu$ ou $\sigma$ ou $\mu$ et $\sigma$. $\xi$ est choisi comme constant. Sur la base bibliographique, on choisi $t_+ = 1985$. Cette année de rupture est motivée par plusieurs considérations : 1) des études précédentes ont montré que les tendances statistiquement significatives des extrêmes de précipitations quotidiennes dans le sud de la France commencent dans les années 1980 [@Blanchet2018] ; 2) c'est en ce point que la log-vraisemblance est maximisée [@Blanchet2018]. En utilisant une année de rupture, nous sommes en mesure de minimiser les potentiels biais découlant des différentes longueurs d'observations tout en pouvant prendre en compte les données supplémentaires antérieures à 1985 pour rendre les estimations plus robustes.

#### Niveau de retour

Le niveau de retour (ou quantile d’ordre $1 - \tfrac{1}{T}$) dans une loi GEV (Figure 3) correspond à une valeur seuil $z_T$ que l’on dépasse, en moyenne, une fois tous les $T$ ans. Soit $X \sim \mathrm{GEV}(\mu, \sigma, \xi)$, alors en notant $F^{-1}$ la fonction quantile de la GEV, on obtient :

:::{.callout}

$$
\mathbb{P}(X > z_T) = \frac{1}{T}, \quad \text{soit} \quad z_T = F^{-1}\left(1 - \frac{1}{T} \right) = 
\begin{cases}
\mu + \frac{\sigma}{\xi} \left[ \left( -\log\left(1 - \frac{1}{T}\right) \right)^{-\xi} - 1 \right] & \text{si } \xi \ne 0 \\
\mu - \sigma \log \left( -\log\left(1 - \frac{1}{T} \right) \right) & \text{si } \xi = 0 \quad \text{(Gumbel)}
\end{cases}
$$

:::

<!-- @berghald2024caracterisation montre l'influence des paramètres de la GEV sur la densité de probabilité et sur la relation entre période et niveau de retour (Figure 3).

\
\begin{center}
  \includegraphics[width=0.6\textwidth]{figures/fig_gev.png}
\end{center}

\captionof{figure}{\small Influence des paramètres de la GEV sur la densité de probabilité (à gauche) et sur la relation entre période et niveau de retour (à droite) : position $\mu$ (en haut), échelle $\sigma$ (centre) et forme $\xi$ (en bas). Les lignes pointillées montrent le niveau de retour de 20\,ans pour les différents cas \cite{berghald2024caracterisation}.} -->

#### Vraisemblance et maximum de vraisemblance

Soit la fonction de vraisemblance ${\displaystyle {\mathcal {L}}(\theta ;x)} : {\displaystyle \theta \mapsto f(x;\theta )}$. La log-vraisemblance $\ell(\theta) = \log \mathcal{L}(\theta)$ s’écrit après développement (Annexes 1-1) :

:::{.callout}

$$
\ell(\theta)=
-\sum_{i=1}^n\Bigl[
\log\sigma
+\Bigl(1+\tfrac1{\xi}\Bigr)\log y_i
+y_i^{-\frac{1}{\xi}}
\Bigr]
\quad \text{avec} \quad  y_i(\theta)=1+\xi\;\frac{x_i-\mu}{\sigma}
\tag{1}
$$

:::

On obtient alors :

$$
\scalebox{0.7}{
\(
\begin{aligned}
\ell_{M_0}(\mu_0, \sigma_0, \xi_0) &=
-\sum_{i=1}^n \left[
\log \sigma_0 +
\left(1 + \frac{1}{\xi_0} \right) \log \left(1 + \xi_0 \frac{x_i - \mu_0}{\sigma_0} \right) +
\left(1 + \xi_0 \frac{x_i - \mu_0}{\sigma_0} \right)^{-1/\xi_0}
\right]\\ 
\ell_{M_1}(\mu_0, \mu_1, \sigma_0, \xi_0) &=
-\sum_{i=1}^n \left[
\log \sigma_0 +
\left(1 + \frac{1}{\xi_0} \right) \log \left(1 + \xi_0 \frac{x_i - (\mu_0 + \mu_1 \cdot \tilde{t}_i)}{\sigma_0} \right) +
\left(1 + \xi_0 \frac{x_i - (\mu_0 + \mu_1 \cdot \tilde{t}_i)}{\sigma_0} \right)^{-1/\xi_0}
\right]\\ 
\ell_{M_2}(\mu_0, \sigma_0, \sigma_1, \xi_0) &=
-\sum_{i=1}^n \left[
\log (\sigma_0 + \sigma_1 \tilde{t}_i) +
\left(1 + \frac{1}{\xi_0} \right) \log \left(1 + \xi_0 \frac{x_i - \mu_0}{\sigma_0 + \sigma_1 \tilde{t}_i} \right) +
\left(1 + \xi_0 \frac{x_i - \mu_0}{\sigma_0 + \sigma_1 \tilde{t}_i} \right)^{-1/\xi_0}
\right]\\ 
\ell_{M_3}(\mu_0, \mu_1, \sigma_0, \sigma_1, \xi_0) &=
-\sum_{i=1}^n \left[
\log (\sigma_0 + \sigma_1 \tilde{t}_i) +
\left(1 + \frac{1}{\xi_0} \right) \log \left(1 + \xi_0 \frac{x_i - (\mu_0 + \mu_1 \tilde{t}_i)}{\sigma_0 + \sigma_1 \tilde{t}_i} \right) +
\left(1 + \xi_0 \frac{x_i - (\mu_0 + \mu_1 \tilde{t}_i)}{\sigma_0 + \sigma_1 \tilde{t}_i} \right)^{-1/\xi_0}
\right]
\end{aligned}
\tag{1'}
\)
}
$$

Les vraisemblances de $M_1^*, M_2^*, M_3^*$ sont obtenues en remplaçant $\tilde{t}_i$ par $\tilde{t}_i^\ast$ dans les expressions ci-dessus.

En pratique, les paramètres $(\mu, \sigma, \xi)$ sont inconnus et estimés à partir des données par un estimateur $\hat{\theta} = (\hat{\mu}, \hat{\sigma}, \hat{\xi})$ obtenu par maximum de vraisemblance (MLE) via une optimisation numérique tel que $\hat{\theta} = \arg\max_{\theta} \, \ell(\theta)$. Il n’existe pas de formule explicite des paramètres, ceux-ci étant déterminés numériquement en maximisant la vraisemblance.

L’estimateur du niveau de retour $\hat{z}_T$ s’écrit alors $\hat{z}_{T}\;=\;F^{-1}_{\hat{\theta}}\!\left(1-\frac{1}{T}\right)$. Le MLE classique donne un point estimé, mais pas d’intervalle. On souhaite aussi connaître l’incertitude autour de l'estimation de $\hat{z}_T$. Pour cela, on utilise la vraisemblance profilée.

#### Vraisemblance profilée et intervalle de confiance d'un niveau de retour

$z_T$ peut se réecrire sous la forme $\mu = z_T - \dfrac{\sigma}{\xi} \left[ \left( -\log\left(1 - \frac{1}{T} \right) \right)^{-\xi} - 1 \right]$.
La combinaison des paramètres temporels de la loi GEV conduit à une expression linéaire en $t$ : 

$$
z_T(t) = z_{T,0} + z_{T,1} \cdot t
$$

En développant les paramètres soumis à un effet temporel et on regroupant terme à terme on peut montrer que (Annexes 1-2) :

$$
\mu_1(z_{T,1}) = z_{T,1} -\dfrac{\hat{\sigma_1}}{\hat{\xi}_0}\Bigl(\bigl[-\log(1-\tfrac1T)\bigr]^{-\hat{\xi}_0}-1\Bigr) \quad \text{et} \quad
\sigma_1(z_{T,1}) = \dfrac{\hat{\xi}_0\,\bigl(z_{T,1}-\hat{\mu_1}\bigr)}{\bigl[-\log\!\bigl(1-\tfrac1T\bigr)\bigr]^{-\hat{\xi}_0}-1}
$$

On cherche l'intervalle de confiance sur $z_{T,1}$ donc pour chaque valeur candidate $z_{T,1}$ dans une grille (autour de l’estimateur $\hat{z}_{T,1}$), on maximise les log-vraisemblances (1') qui deviennent des log-vraisemblances profilées $\ell^{\,p}$ :

$$
\scalebox{0.7}{
\(
\begin{aligned}
\underset{\hat{\sigma}_1 = 0}{\ell_{M_1}^{\,p}(z_{T,1} \ ; \hat{\mu}_0, \hat{\sigma}_0, \hat{\xi}_0)} &= 
-\sum_{i=1}^n \left[
\log \hat{\sigma}_0 +
\left(1 + \frac{1}{\hat{\xi}_0} \right) \log \left(1 + \hat{\xi}_0 \frac{x_i - (\hat{\mu}_0 + \mu_1(z_{T,1}) \cdot \tilde{t}_i)}{\hat{\sigma}_0} \right) +
\left(1 + \hat{\xi}_0 \frac{x_i - (\hat{\mu}_0 + \mu_1(z_{T,1}) \cdot \tilde{t}_i)}{\hat{\sigma}_0} \right)^{-1/\hat{\xi}_0}
\right]\\
\underset{\hat{\mu}_1 = 0}{\ell_{M_2}^{\,p}(z_{T,1} \ ; \hat{\mu}_0, \hat{\sigma}_0, \hat{\xi}_0)} &= 
-\sum_{i=1}^n \left[
\log (\hat{\sigma}_0 + \sigma_1(z_{T,1}) \cdot \tilde{t}_i) +
\left(1 + \frac{1}{\hat{\xi}_0} \right) \log \left(1 + \hat{\xi}_0 \frac{x_i - \hat{\mu}_0}{\hat{\sigma}_0 + \sigma_1(z_{T,1}) \cdot \tilde{t}_i} \right) +
\left(1 + \hat{\xi}_0 \frac{x_i - \hat{\mu}_0}{\hat{\sigma}_0 + \sigma_1(z_{T,1}) \cdot \tilde{t}_i} \right)^{-1/\hat{\xi}_0}
\right]\\
\ell_{M_3}^{\,p}(z_{T,1} \ ; \hat{\mu}_0, \hat{\sigma}_0, \hat{\sigma}_1, \hat{\xi}_0) &= 
-\sum_{i=1}^n \left[
\log (\hat{\sigma}_0 + \hat{\sigma}_1 \tilde{t}_i) +
\left(1 + \frac{1}{\hat{\xi}_0} \right) \log \left(1 + \hat{\xi}_0 \frac{x_i - (\hat{\mu}_0 + \mu_1(z_{T,1}) \tilde{t}_i)}{\hat{\sigma}_0 + \hat{\sigma}_1 \tilde{t}_i} \right) +
\left(1 + \hat{\xi}_0 \frac{x_i - (\hat{\mu}_0 + \mu_1(z_{T,1}) \tilde{t}_i)}{\hat{\sigma}_0 + \hat{\sigma}_1 \tilde{t}_i} \right)^{-1/\hat{\xi}_0}
\right]\\
\end{aligned}
\tag{2}
\)
}
$$

On cherche donc :

$$
\hat{z}_{T,1} = \underset{z_{T,1}}{\arg\max} \; \ell_{M_\bullet}^{\,p}(z_{T,1} \ ; \hat{\theta}_{\bullet}) \quad \text{avec} \quad
\hat{\theta}_{\bullet} = \begin{cases}
\hat{\theta}_{1}^{\,p} = (\hat{\mu_0}, \hat{\sigma_0}, \hat{\xi_0}) & \text{pour } M_1 \\
\hat{\theta}_2^{\,p} = (\hat{\mu_0}, \hat{\sigma_0}, \hat{\xi_0}) & \text{pour } M_2 \\
\hat{\theta}_3^{\,p} = (\hat{\mu_0}, \hat{\sigma_0}, \hat{\sigma_1}, \hat{\xi_0}) & \text{pour } M_3 \\
\end{cases}
$$

On trace ainsi pour chaque modèle $M_\bullet$ la fonction ${\displaystyle \mathcal{L}_{M_\bullet} : z_{T,1} {\mapsto} \ell_{M_\bullet}^{\,p}(z_{T,1} \ ; \hat{\theta}_{\bullet})}$

L’intervalle de confiance de $\hat{z}_{T,1}$ pour un modèle $M_\bullet$ au seuil $(1 - \alpha)$ basé sur le profil de vraisemblance est donné par :

$$
\operatorname{IC}_{M_\bullet}^{(1-\alpha)}\!\bigl(\hat{z}_{T,1}\bigr)
   = \Bigl\{\, z_{T,1}\;:\;
        2\bigl[\ell_{M_\bullet}^{\,p}(\hat{z}_{T,1} \ ; \hat{\theta}_{\bullet})-\ell_{M_\bullet}^{\,p}(z_{T,1} \ ; \hat{\theta}_{\bullet})\bigr]
        \le \chi^{2}_{1,\,1-\alpha} \Bigr\}
$$

où $\chi^2_{1,1-\alpha}$ est le quantile d’ordre $1 - \alpha$ d’une loi du $\chi^2$ à un degré de liberté. On fixe ici $\alpha=0{,}10$. Lorsque l'intervalle de confiance ne contient pas 0 alors $\hat{z}_{T,1}$ est significatif.

### Choix du meilleur modèle

#### Test du rapport de vraisemblance (LRT)

En tout point géographique on dispose de $M_0, M_1, M_2, M_3, M_1^\ast, M_2^\ast \text{ et } M_3^\ast$. Notons $k_j$ le nombre de paramètres du modèle $M_j$. On souhaite tester, pour chaque modèle non stationnaire $j \neq 0$, l’hypothèse nulle $H_0$ : $M_j$ ne fait pas mieux que $M_0$ (stationnarité). Pour chaque modèle non stationnaire $j\neq 0$ et chaque point $i$ en notant $p_{ij}$ la $p$-valeur :

$$
\Lambda_{ij}=2\bigl(\ell_{ij}-\ell_{i0}\bigr) \overset{H_0}{\sim}\chi^{2}_{\;k_j-k_0} \quad \text{avec} \quad p_{ij}= \mathbb{P}(\chi^{2}_{k_j-k_0}\ge \Lambda_{ij}\bigr)
$$

#### Règle hiérarchique de sélection

Soit $\alpha=0{,}10$ le seuil d’intérêt. Si un des deux modèles $M_3$ ou $M_3^\ast$ vérifie $p_{ij}\le\alpha$, on retient $j=\arg\min_{j\in\{3,3^\ast\}} p_{ij}$. Sinon, on compare l’ensemble des six modèles non stationnaires et l’on sélectionne $j=\arg\min_{j\in\{1,1^\ast,2,2^\ast,3,3^\ast\}} p_{ij}$. Cette double étape privilégie les formes *simultanément* temporelles sur $\mu$ et $\sigma$ quand elles sont statistiquement justifiées. Cela assure que la complexité n’est introduite que lorsqu’elle apporte une information statistiquement crédible tout en livrant, pour chaque poste, un modèle non stationnaire.

### Calcul des tendances

A partir des niveaux de retour 10 ans ($T = 10$) la tendance relative (en %) entre 1995 et 2022 est calculée via la formule ci-dessous. Un exemple graphique est disponible en annexes 2-6.

:::{.callout}

$$
\text{Tendance} = \frac{z_T^{2022} - z_T^{1995}}{z_T^{1995}} \cdot {100}
$$

:::

### Concordance

On évalue l'accord entre les statistiques descriptives et les tendances obtenues à partir des simulations du modèle AROME et celles observées dans la réalité. Pour ce faire, chaque station Météo-France est associée au point de grille AROME (2,5 km × 2,5 km) correspondant à sa localisation géographique. Cette correspondance permet de calculer la corrélation de Pearson (*r*) ainsi que l'erreur moyenne (*ME*) entre les valeurs observées et simulées.

### Représentation cartographique

Pour homogénéiser les amplitudes extrêmes entre jeux de données (AROME et données observées), on applique une saturation de couleur suivant la période étudiée. Soit une période $P \in \{\text{HYDRO}, \text{DJF},\text{MAM},\text{JJA},\text{SON}\}$ ou $P \in \{\text{JAN},\text{FEV},\text{MAR},\text{...},\text{OCT},\text{NOV},\text{DEC}\}$. Pour chaque statistique (nombre de jour de pluie, cumul, moyenne des maxima, tendance relative) $T_j$ avec $j \in P$, on calcule le *q*-ième percentile des valeurs absolues :

$$
s_j(P)=\operatorname{Quantile}_{q}\big(|T_j|\big)
$$

Le seuil commun de la période $P$ est défini par $S(P)=\max_j s_j(P)$. On remplace ensuite, pour toute valeur $x\in T_j$, $x \;\leftarrow\; \operatorname{sign}(x)\,\min\big(|x|,\,S(P)\big)$.

On fixe $q$ de la manière suivante :

- Pour le nombre de jour de pluie : $q = 99,9$ pour l'échelle journalière 1959-2022, 1990-2022 et l'échelle horaire 1990-2022.
- Le cumul et la moyenne des maxima de précipitations : $q = 99,0$ pour l'échelle journalière 1959-2022, 1990-2022 et l'échelle horaire 1990-2022.
- La tendance relative : $q = 99,0$ pour l'échelle journalière 1959-2022 et 1990-2022 et $q = 90,0$ pour l'échelle horaire 1990-2022.

Les courbes de niveaux 400 et 800m sont représentées en trait fin.


## Résultats

Nous commençons par évaluer, point de grille par point de grille, la capacité d’AROME à reproduire le régime de précipitations observé par les stations Météo‑France. Cette première étape stationnaire sert d'évaluation de qualité avant d'étudier les tendances des niveaux de retour.

### Évaluation de la climatologie des précipitations simulées par AROME

Afin d’apprécier la capacité d’AROME à restituer la climatologie des précipitations, différents indicateurs sont confrontées aux observations de stations pour une saison donnée : 1) le nombre de jours de pluie (seuil ≥ 1 mm/j) ; 2) le cumul des précipitations ; et 3) la moyenne des maxima des précipitations.

In [ ]:
jour_pluie = assemble(
  "../outputs/maps/stats_numday/quotidien/compare_1/sat_99.9/hydro/mod_norast.svg",
  "../outputs/maps/stats_numday/quotidien/compare_1/sat_99.9/hydro/obs_norast.svg",
  "../outputs/maps/stats_numday/quotidien/compare_1/sat_99.9/legend.svg",
  "figures/jour_pluie.svg"
)
df_jour_pluie = combined.loc[(combined["echelle"] == "quotidien") & (combined["season"] == "hydro") & (combined["col_calculate"] == "numday"), ["n", "r", "me", "delta"]]

mean_pluie_jour = assemble(
  "../outputs/maps/stats_mean/quotidien/compare_1/sat_99.0/hydro/mod_norast.svg",
  "../outputs/maps/stats_mean/quotidien/compare_1/sat_99.0/hydro/obs_norast.svg",
  "../outputs/maps/stats_mean/quotidien/compare_1/sat_99.0/legend.svg",
  "figures/mean_pluie_jour.svg"
)
df_mean_pluie_jour = combined.loc[(combined["echelle"] == "quotidien") & (combined["season"] == "hydro") & (combined["col_calculate"] == "mean"), ["n", "r", "me", "delta"]]


mean_max_pluie_jour = assemble(
  "../outputs/maps/stats_mean-max/quotidien/compare_9/sat_99.0/hydro/mod_norast.svg",
  "../outputs/maps/stats_mean-max/quotidien/compare_9/sat_99.0/hydro/obs_norast.svg",
  "../outputs/maps/stats_mean-max/quotidien/compare_9/sat_99.0/legend.svg",
  "figures/mean-max_pluie_jour.svg"
)
df_mean_max_pluie_jour = combined.loc[(combined["echelle"] == "quotidien") & (combined["season"] == "hydro") & (combined["col_calculate"] == "mean-max"), ["n", "r", "me", "delta"]]

mean_max_pluie_horaire = assemble(
  "../outputs/maps/stats_mean-max/horaire/compare_9/sat_99.0/hydro/mod_norast.svg",
  "../outputs/maps/stats_mean-max/horaire/compare_9/sat_99.0/hydro/obs_norast.svg",
  "../outputs/maps/stats_mean-max/horaire/compare_9/sat_99.0/legend.svg",
  "figures/mean-max_pluie_horaire.svg"
)

df_mean_max_pluie_horaire = combined.loc[(combined["echelle"] == "horaire") & (combined["season"] == "hydro") & (combined["col_calculate"] == "mean-max"), ["n", "r", "me", "delta"]]

df_mean_max_pluie_horaire_jja = combined.loc[(combined["echelle"] == "horaire") & (combined["season"] == "jja") & (combined["col_calculate"] == "mean-max"), ["n", "r", "me", "delta"]]

#### Distribution spatiale

La distribution spatiale des indicateurs d'AROME au regard des observations permet de : 1) valider et diagnostiquer le modèle en localisant les biais et vérifier la structure spatiale (gradients et effets orographiques) ; et 2) comprendre les processus physique (distinguer les régimes (méditerranéen, alpin, plaine) et voir où AROME surestime l’orographique ou sous-estime les convectifs intenses).

##### Nombre de jours de précipitations

Sur la période 1959-2022 (Figure 4, colonne de gauche), AROME surestime légèrement la fréquence annuelle des jours de précipitation : `{python} f'{df_jour_pluie['me'].iloc[0].item():+.2f}'` jours/an (soit `{python} f'{df_jour_pluie['delta'].iloc[0].item():+.2f}'`%). Des biais locaux marqués subsistent toutefois : plus de +30 jours pour plusieurs stations du Massif central et des Pyrénées, tandis que des biais négatifs (−10 à −30 jours) se concentrent dans les Alpes du Nord, le Finistère et l’Alsace. La corrélation spatiale est élevée (*r =* `{python} f'{df_jour_pluie['r'].iloc[0].item():.2f}'`), indiquant que le modèle reproduit correctement les grands gradients régionaux. AROME et les observations s’accordent pour situer les fréquences maximales sur les massifs montagneux (Alpes, Pyrénées, Massif central, Vosges, Jura), avec plus de 140–160 jours/an, et des valeurs également élevées sur le Grand Ouest atlantique (Bretagne, Normandie, Pays de la Loire : 80–120 jours/an). À l’inverse, la façade méditerranéenne et le pourtour provençal restent les plus secs en fréquence, souvent < 50–70 jours/an.

##### Cumul annuel des précipitations

Sur la période 1959-2022 (Figure 4, colonne du centre), AROME estime correctement le cumul annuel des précipitations : `{python} f'{df_mean_pluie_jour['me'].iloc[0].item():+.2f}'` mm/an (soit `{python} f'{df_mean_pluie_jour['delta'].iloc[0].item():+.2f}'`%). Des biais locaux marqués subsistent toutefois : plus de +1,0 mm/j pour l'ensemble des stations de la crête pyrénéenne et de la station de Chamonix, tandis que des biais négatifs (−0,5 à −1,5 mm/j) se concentrent dans les Alpes du Nord, la côte basque et l’Alsace. La corrélation spatiale est élevée (*r =* `{python} f'{df_mean_pluie_jour['r'].iloc[0].item():.2f}'`), indiquant que le modèle reproduit correctement les grands gradients régionaux. La bordure atlantique sud‑ouest (Pyrénées, Aquitaine) et les Alpes du Nord reçoivent les plus forts cumuls (plus de 5 mm/j). Le Massif central et les reliefs intérieurs (Vosges, Jura) présentent des cumuls intermédiaires (2,5-4 mm/j). Le pourtour méditerranéen (Languedoc, Provence) reste globalement plus sec (< 1,5 mm/j), malgré des pluies intenses ponctuelles. Ces résultats sont identiques avec les données horaires de 1990 à 2022 (Annexe 2-2.3).


\setlength{\tabcolsep}{0pt}  

| \shortstack{\small Nombre de jours\\ par an\\ de précipitations} | \shortstack{\small Cumul\\ annuel\\ des précipitations} | \shortstack{\small Moyenne des maxima\\ journaliers\\ des précipitations} | \shortstack{\small Moyenne des maxima\\ horaires\\ des précipitations} |
|:---:|:---:|:---:|:---:|
| ![](figures/jour_pluie.pdf) | ![](figures/mean_pluie_jour.pdf) | ![](figures/mean-max_pluie_jour.pdf) | ![](figures/mean-max_pluie_horaire.pdf) |
| \small $r =$ `{python} f'{df_jour_pluie['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_jour_pluie['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_mean_pluie_jour['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_mean_pluie_jour['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_mean_max_pluie_jour['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_mean_max_pluie_jour['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_mean_max_pluie_horaire['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_mean_max_pluie_horaire['n'].iloc[0].item()}'`) |
| ![](../outputs/maps/stats_numday/quotidien/compare_1/sat_99.9/hydro/obs_rast_diff.pdf) | ![](../outputs/maps/stats_mean/quotidien/compare_1/sat_99.0/hydro/obs_rast_diff.pdf) | ![](../outputs/maps/stats_mean-max/quotidien/compare_9/sat_99.0/hydro/obs_rast_diff.pdf) | ![](../outputs/maps/stats_mean-max/horaire/compare_9/sat_99.0/hydro/obs_rast_diff.pdf) |
| \small $ME =$ `{python} f'{df_jour_pluie['me'].iloc[0].item():+.2f}'` j \ (`{python} f'{df_jour_pluie['delta'].iloc[0].item():+.2f}'`%) | \small $ME =$ `{python} f'{df_mean_pluie_jour['me'].iloc[0].item():+.2f}'` mm/an (`{python} f'{df_mean_pluie_jour['delta'].iloc[0].item():+.2f}'`%) | \small $ME =$ `{python} f'{df_mean_max_pluie_jour['me'].iloc[0].item():+.2f}'` mm/j (`{python} f'{df_mean_max_pluie_jour['delta'].iloc[0].item():+.2f}'`%) | \small $ME =$ `{python} f'{df_mean_max_pluie_horaire['me'].iloc[0].item():+.2f}'` mm/j (`{python} f'{df_mean_max_pluie_horaire['delta'].iloc[0].item():+.2f}'`%) |


\captionof{figure}{\small Climatologie entre le modèle AROME (première ligne), les stations Météo-France (deuxième ligne) avec la corrélation ($r$) et le nombre de stations comparés (n) ainsi que la différence AROME-Station (troisième ligne) avec le biais ($ME$) et l'écart relatif (\%) associé issues de données journalières allant de 1959 à 2022 pour une année hydrologique.}


##### Moyenne des maxima des précipitations

Sur la période 1959-2022 (Figure 4, colonne de droite), AROME sous-estime légèrement la moyenne des maxima journaliers des précipitations : `{python} f'{df_mean_max_pluie_jour['me'].iloc[0].item():+.2f}'` mm/j (soit `{python} f'{df_mean_max_pluie_jour['delta'].iloc[0].item():+.2f}'`%). Il subsiste d'importants biais locaux : déficits de −5 à moins de −20 mm/j sur l'ensemble des Cévennes et se prolongeant vers l'extrimité Nord des Alpes du Nord, sur la côté sud-est et la côte basque ; jusqu'à -5 mm/j sur une large partie du centre de la France et de l'Alsace ; de +5 à +20 mm/j sur la crête pyrénéennes, les contreforts ouest des Cévennes et les Alpes du Nord. La corrélation spatiale est élevée (*r =* `{python} f'{df_mean_max_pluie_jour['r'].iloc[0].item():.2f}'`), indiquant que le modèle reproduit correctement les grands gradients régionaux. Les plus fortes précipitations quotidiennes moyennes se rencontrent dans les Cévennes et plus généralement sur la face sud-est du Massif central (environ 100–125 mm/j). Les reliefs alpins et pyrénéens montrent aussi des maxima élevés (80–100 mm/j). La façade atlantique et le bassin parisien présentent des maxima plus modérés (30–60 mm/j), tandis que la Provence et la Côte d’Azur, malgré une fréquence moindre, peuvent localement connaître de très gros orages (40–80 mm/j en moyenne).

\

Ces premiers résultats, marqués par des biais locaux importants sur les maxima journaliers, nous conduisent à descendre au pas de temps horaire (Figure 4). Sur la période 1990-2022, AROME sous-estime fortement la moyenne des maxima horaires des précipitations : `{python} f'{df_mean_max_pluie_horaire['me'].iloc[0].item():+.2f}'` mm/h (soit `{python} f'{df_mean_max_pluie_horaire['delta'].iloc[0].item():+.2f}'`%). Il subsiste d'importants biais : déficits de −5 à moins de −10 mm/h sur presque l'ensemble de la France mais surtout dans les Cévennes et la vallée du Rhône. La corrélation spatiale reste élevée (*r =* `{python} f'{df_mean_max_pluie_horaire['r'].iloc[0].item():.2f}'`), attestant d’une bonne restitution des grands gradients déjà observés au pas journalier. Au regard des saisons (Annexes 2-3.3), le biais s'étend de -0,02 mm/h (DJF) à -3,75 mm/h (JJA), suggérant une sous‑représentation marquée des extrêmes convectifs estivaux.

#### Corrélation entre le modèle AROME et les stations Météo-France

Quel que soit l'indicateur (Figure 6), le modèle AROME reproduit fidèlement les observations, avec une corrélation minimale de 0,70.
Indépendamment de l’échelle temporelle retenue — journalière (1959‑2022 ou 1990‑2022) ou horaire (1990‑2022) — et de la saison, les champs simulés s’accordent très bien avec les données mesurées : la corrélation varie entre 0,92 et 0,98 pour le nombre de jours de pluie et le cumul des précipitations.

Cette performance se maintient pour la moyenne des maxima journaliers (périodes 1959‑2022 et 1990‑2022) sur l’année hydrologique, l’automne, l’hiver et le printemps, mais elle se dégrade en été, avec une corrélation de 0,85. À l’échelle horaire, la qualité de l’estimation des maxima se détériore encore : la corrélation baisse de 0,4 à 0,8 point selon la saison pour l’année hydrologique, l’automne et l’hiver, et chute à 0,70 au printemps et en été. 

In [ ]:
plot_r_by_season(combined, echelles=['quotidien', 'quotidien_reduce', 'horaire'], col_values=['numday', 'mean', 'mean-max'], nom_fichier="histo_numday_mean_mean-max")

![](figures/histo_numday_mean_mean-max.png){width=100%}

\captionof{figure}{\small Corrélations des données climatologiques entre le modèle AROME et les stations Météo-France pour chacune des sources de données.}

::: {.callout}
AROME sous‑représente les extrêmes convectifs estivaux, surtout à l’échelle horaire, malgré une structure spatiale bien captée.
:::

### Evaluation des tendances des précipitations extrêmes

Nous évaluons à présent les tendances des niveaux de retour décennaux (T = 10 ans) *significatifs* des précipitations, en visualisant leur hétérogénéité spatiale et saisonnière et en quantifiant leur amplitude et leur significativité statistique afin d’identifier les régions et périodes où l’intensification des extrêmes est la plus marquée.

#### Distribution spatiale

Comme précédemment on commence par analyser la distribution spatiale des tendances d'AROME au regard des observations.

##### Données journalières

Nous commençons par vérifier que notre méthodologie restitue bien les tendances déjà établies à l’échelle journalière sur 1959‑2022 — période longue, largement documentée — de façon à valider les signaux avant de descendre au pas horaire. 

Les tendances relatives de 1995 à 2022 estimées avec AROME sont spatialement hétérogènes (Annexes 2‑4.1.1.2, **HYDRO**). Seul le Mercantour se distingue par une hausse significative comprise entre +20 et +30 %. Les stations montrent par ailleurs un signal positif généralisé le long de la vallée du Rhône, avec des tendances de +5 à >+30 %. Hors de ces zones, les tendances sont faibles, de signe variable et le plus souvent non significatives, ce qui plaide pour une analyse saisonnière (Figure 7).

In [ ]:
trend_pluie_hydro = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/hydro/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/hydro/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_hydro.svg"
)
df_trend_pluie_hydro = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "hydro") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_son = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/son/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/son/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_son.svg"
)
df_trend_pluie_son = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "son") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_djf = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/djf/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/djf/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_djf.svg"
)
df_trend_pluie_djf = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "djf") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_mam = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/mam/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/mam/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_mam.svg"
)
df_trend_pluie_mam = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "mam") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_jja = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/jja/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/jja/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_jja.svg"
)
df_trend_pluie_jja = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "jja") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_ond = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/ond/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/ond/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_ond.svg"
)
df_trend_pluie_ond = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "ond") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_jfm = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/jfm/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/jfm/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_jfm.svg"
)
df_trend_pluie_jfm = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "jfm") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_amj = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/amj/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/amj/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_amj.svg"
)
df_trend_pluie_amj = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "amj") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_jas = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/jas/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/jas/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_9/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_jas.svg"
)
df_trend_pluie_jas = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "jas") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

\newpage
\newgeometry{left=0.2cm,right=0.2cm,top=0.2cm,bottom=0.2cm}

\setlength{\tabcolsep}{0pt}  

| | | |
| :---: | :---: | :---: |
| \small \textbf{HYDRO} | \small \textbf{SON} | \small \textbf{DJF} | 
| \small $r =$ `{python} f'{df_trend_pluie_hydro['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_hydro['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_pluie_son['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_son['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_pluie_djf['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_djf['n'].iloc[0].item()}'`) |
| \small $ME =$ `{python} f'{df_trend_pluie_hydro['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_pluie_son['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_pluie_djf['me'].iloc[0].item():+.2f}'`% |
| ![](figures/trend_pluie_hydro.pdf) | ![](figures/trend_pluie_son.pdf) | ![](figures/trend_pluie_djf.pdf) |
\small \textbf{MAM} | \small \textbf{JJA} | \small \textbf{OND} |
|  \small $r =$ `{python} f'{df_trend_pluie_mam['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_mam['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_pluie_jja['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_jja['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_pluie_ond['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_ond['n'].iloc[0].item()}'`) | 
|  \small $ME =$ `{python} f'{df_trend_pluie_mam['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_pluie_jja['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_pluie_ond['me'].iloc[0].item():+.2f}'`% | 
|  ![](figures/trend_pluie_mam.pdf) | ![](figures/trend_pluie_jja.pdf) | ![](figures/trend_pluie_ond.pdf) | 
 \small \textbf{JFM} | \small \textbf{AMJ} | \small \textbf{JAS} |
| \small $r =$ `{python} f'{df_trend_pluie_jfm['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_jfm['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_pluie_amj['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_amj['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_pluie_jas['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_pluie_jas['n'].iloc[0].item()}'`) |
| \small $ME =$ `{python} f'{df_trend_pluie_jfm['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_pluie_amj['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_pluie_jas['me'].iloc[0].item():+.2f}'`% |
| ![](figures/trend_pluie_jfm.pdf) | ![](figures/trend_pluie_amj.pdf) | ![](figures/trend_pluie_jas.pdf) |


\captionof{figure}{\small Analyse saisonnière des tendances relatives de 1995 à 2022 (\%) du niveau de retour 10 ans significatif entre le modèle AROME (première ligne) et les stations Météo-France (deuxième ligne) avec la corrélation ($r$), le nombre de stations comparés (n) et le biais ($ME$) issues des maxima de précipitations au pas de temps journalier allant de 1959 à 2022.}

\restoregeometry
\

À l’automne (**SON**), AROME indique des tendances négatives (−10 à < −30 %) sur le bassin parisien, le versant occidental du Massif central et les pré‑Alpes, et une hausse marquée dans le Mercantour, comme précédemment. Les stations confirment ce schéma, mais avec une diminution moins prononcée sur les pré‑Alpes et un signal haussier plus net sur les Alpes du Nord. La vallée du Rhône se distingue par une augmentation pouvant atteindre localement +40%. En hiver (**DJF**), AROME met en évidence des tendances négatives marquées — jusqu’à −40 % — depuis les Pyrénées orientales jusqu’à la haute crête cévenole ; des diminutions plus modérées (−15 %) affectent également les Alpes du Nord et le Jura. À l’inverse, les façades atlantique nord et manche présentent des tendances positives de +10 à +30%. Les stations confirment ce schéma, tout en étendant la zone de baisse sur les Alpes du Nord. Au printemps (**MAM**), le signal est peu structuré : AROME n’isole pas de pattern net, mais suggère des tendances positives sur la moitié nord et ouest de la France (+15 à +30%). Les stations renforcent ce diagnostic, avec une hausse généralisée sur l’ensemble de la moitié Nord, localement > +35%. En été (**JJA**), AROME met en évidence une baisse généralisée sur la moitié sud de la France, atteignant −10 à <−30% le long de l’arc méditerranéen, avec un noyau de diminution marqué sur les bassins pyrénéens orientaux. Les stations confirment ce motif spatial. \

Globalement, AROME reproduit la distribution spatiale des tendances observées, mais avec une faible corrélation et une extension parfois réduite — sur les Alpes du Nord en automne et en hiver, ainsi que sur la moitié nord de la France au printemps — et sous‑estime systématiquement leur amplitude, avec un biais moyen compris entre −2,42% (MAM) et −4,87% (HYDRO).

##### Données horaires

Nous passons désormais à l’échelle horaire afin d’évaluer l’éventuelle intensification des extrêmes infra‑journaliers. Face à l’hétérogénéité spatiale persistante des tendances saisonnières (Annexes 2‑4.3.1.2, HYDRO, SON, DJF, MAM et JJA), nous affinons l’analyse au pas mensuel (Figure 8).


In [ ]:
trend_horaire_pluie_hydro = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/hydro/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/hydro/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_hydro.svg"
)
df_trend_horaire_pluie_hydro = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "hydro") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_son = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/son/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/son/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_son.svg"
)
df_trend_horaire_pluie_son = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "son") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_djf = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/djf/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/djf/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_djf.svg"
)
df_trend_horaire_pluie_djf = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "djf") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_mam = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/mam/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/mam/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_mam.svg"
)
df_trend_horaire_pluie_mam = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "mam") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_jja = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/jja/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/jja/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_jja.svg"
)
df_trend_horaire_pluie_jja = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "jja") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_ond = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/ond/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/ond/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_ond.svg"
)
df_trend_horaire_pluie_ond = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "ond") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_jfm = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/jfm/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/jfm/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_jfm.svg"
)
df_trend_horaire_pluie_jfm = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "jfm") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_amj = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/amj/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/amj/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_amj.svg"
)
df_trend_horaire_pluie_amj = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "amj") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_horaire_pluie_jas = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/jas/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/jas/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_9/sat_90.0/legend_signif.svg",
  "figures/trend_horaire_pluie_jas.svg"
)
df_trend_horaire_pluie_jas = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "jas") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

\newpage
\newgeometry{left=0.2cm,right=0.2cm,top=0.2cm,bottom=0.2cm}

\setlength{\tabcolsep}{0pt}  

| | | |
| :---: | :---: | :---: |
| \small \textbf{HYDRO} | \small \textbf{SON} | \small \textbf{DJF} | 
| \small $r =$ `{python} f'{df_trend_horaire_pluie_hydro['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_hydro['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_pluie_son['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_son['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_pluie_djf['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_djf['n'].iloc[0].item()}'`) |
| \small $ME =$ `{python} f'{df_trend_horaire_pluie_hydro['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_pluie_son['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_pluie_djf['me'].iloc[0].item():+.2f}'`% |
| ![](figures/trend_horaire_pluie_hydro.pdf) | ![](figures/trend_horaire_pluie_son.pdf) | ![](figures/trend_horaire_pluie_djf.pdf) |
\small \textbf{MAM} | \small \textbf{JJA} | \small \textbf{OND} |
|  \small $r =$ `{python} f'{df_trend_horaire_pluie_mam['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_mam['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_pluie_jja['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_jja['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_pluie_ond['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_ond['n'].iloc[0].item()}'`) | 
|  \small $ME =$ `{python} f'{df_trend_horaire_pluie_mam['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_pluie_jja['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_pluie_ond['me'].iloc[0].item():+.2f}'`% | 
|  ![](figures/trend_horaire_pluie_mam.pdf) | ![](figures/trend_horaire_pluie_jja.pdf) | ![](figures/trend_horaire_pluie_ond.pdf) | 
 \small \textbf{JFM} | \small \textbf{AMJ} | \small \textbf{JAS} |
| \small $r =$ `{python} f'{df_trend_horaire_pluie_jfm['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_jfm['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_pluie_amj['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_amj['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_pluie_jas['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_pluie_jas['n'].iloc[0].item()}'`) |
| \small $ME =$ `{python} f'{df_trend_horaire_pluie_jfm['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_pluie_amj['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_pluie_jas['me'].iloc[0].item():+.2f}'`% |
| ![](figures/trend_horaire_pluie_jfm.pdf) | ![](figures/trend_horaire_pluie_amj.pdf) | ![](figures/trend_horaire_pluie_jas.pdf) |


\captionof{figure}{\small Analyse saisonnière des tendances relatives de 1995 à 2022 (\%) du niveau de retour 10 ans significatif entre le modèle AROME (première ligne) et les stations Météo-France (deuxième ligne) avec la corrélation ($r$), le nombre de stations comparés (n) et le biais ($ME$) issues des maxima de précipitations au pas de temps horaire allant de 1990 à 2022.}

\restoregeometry
\



#### MOIS

In [ ]:
trend_pluie_quotidien_fev = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/fev/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/fev/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_quotidien_fev.svg"
)
df_trend_quotidien_pluie_fev = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "fev") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_quotidien_mar = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/mar/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/mar/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_quotidien_mar.svg"
)
df_trend_quotidien_pluie_mar = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "mar") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_quotidien_jui = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/nov/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/nov/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_quotidien_jui.svg"
)
df_trend_quotidien_pluie_jui = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "jui") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_quotidien_nov = assemble(
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/nov/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/nov/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/quotidien/compare_12/sat_99.0/legend_signif.svg",
  "figures/trend_pluie_quotidien_nov.svg"
)
df_trend_quotidien_pluie_nov = combined_all.loc[(combined_all["echelle"] == "quotidien") & (combined_all["season"] == "nov") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

\newpage
\newgeometry{left=0.2cm,right=0.2cm,top=0.2cm,bottom=0.2cm}

\setlength{\tabcolsep}{0pt}  

| | | |
| :---: | :---: | :---: | :---: |
| \small \textbf{FEV} | \small \textbf{MAR} | \small \textbf{JUI} | \small \textbf{NOV} | 
| \small $r =$ `{python} f'{df_trend_quotidien_pluie_fev['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_quotidien_pluie_fev['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_quotidien_pluie_mar['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_quotidien_pluie_mar['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_quotidien_pluie_jui['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_quotidien_pluie_jui['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_quotidien_pluie_nov['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_quotidien_pluie_nov['n'].iloc[0].item()}'`) |
| \small $ME =$ `{python} f'{df_trend_quotidien_pluie_fev['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_quotidien_pluie_mar['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_quotidien_pluie_jui['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_quotidien_pluie_nov['me'].iloc[0].item():+.2f}'`% |
| ![](figures/trend_quotidien_pluie_fev.pdf) | ![](figures/trend_quotidien_pluie_mar.pdf) | ![](figures/trend_quotidien_pluie_jui.pdf) | ![](figures/trend_quotidien_pluie_nov.pdf) |

\captionof{figure}{\small Analyse saisonnière des trendances relatives de 1995 à 2022 (\%) du niveau de retour 10 ans significatif entre le modèle AROME (première ligne) et les stations Météo-France (deuxième ligne) avec la corrélation ($r$), le nombre de stations comparés (n) et le biais ($ME$) issues des maxima de précipitations au pas de temps journalier allant de 1959 à 2022.}

\restoregeometry
\


In [ ]:
trend_pluie_horaire_fev = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/fev/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/fev/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/legend_signif.svg",
  "figures/trend_pluie_horaire_fev.svg"
)
df_trend_horaire_pluie_fev = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "fev") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_horaire_mar = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/mar/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/mar/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/legend_signif.svg",
  "figures/trend_pluie_horaire_mar.svg"
)
df_trend_horaire_pluie_mar = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "mar") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_horaire_jui = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/nov/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/nov/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/legend_signif.svg",
  "figures/trend_pluie_horaire_jui.svg"
)
df_trend_horaire_pluie_jui = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "jui") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

trend_pluie_horaire_nov = assemble(
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/nov/mod_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/nov/obs_signif_norast.svg",
  "../outputs/maps/gev_z_T_p/horaire/compare_12/sat_90.0/legend_signif.svg",
  "figures/trend_pluie_horaire_nov.svg"
)
df_trend_horaire_pluie_nov = combined_all.loc[(combined_all["echelle"] == "horaire") & (combined_all["season"] == "nov") & (combined_all["col_calculate"] == "z_T_p_signif"), ["n", "r", "me", "delta"]]

\newpage
\newgeometry{left=0.2cm,right=0.2cm,top=0.2cm,bottom=0.2cm}

\setlength{\tabcolsep}{0pt}  

| | | |
| :---: | :---: | :---: | :---: |
| \small \textbf{FEV} | \small \textbf{MAR} | \small \textbf{JUI} | \small \textbf{NOV} | 
| \small $r =$ `{python} f'{df_trend_horaire_horaire_pluie_fev['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_horaire_pluie_fev['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_horaire_pluie_mar['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_horaire_pluie_mar['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_horaire_pluie_jui['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_horaire_pluie_jui['n'].iloc[0].item()}'`) | \small $r =$ `{python} f'{df_trend_horaire_horaire_pluie_nov['r'].iloc[0].item():.2f}'` (n = `{python} f'{df_trend_horaire_horaire_pluie_nov['n'].iloc[0].item()}'`) |
| \small $ME =$ `{python} f'{df_trend_horaire_horaire_pluie_fev['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_horaire_pluie_mar['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_horaire_pluie_jui['me'].iloc[0].item():+.2f}'`% | \small $ME =$ `{python} f'{df_trend_horaire_horaire_pluie_nov['me'].iloc[0].item():+.2f}'`% |
| ![](figures/trend_horaire_pluie_fev.pdf) | ![](figures/trend_horaire_pluie_mar.pdf) | ![](figures/trend_horaire_pluie_jui.pdf) | ![](figures/trend_horaire_pluie_nov.pdf) |

\captionof{figure}{\small Analyse saisonnière des trendances relatives de 1995 à 2022 (\%) du niveau de retour 10 ans significatif entre le modèle AROME (première ligne) et les stations Météo-France (deuxième ligne) avec la corrélation ($r$), le nombre de stations comparés (n) et le biais ($ME$) issues des maxima de précipitations au pas de temps horaire allant de 1990 à 2022.}

\restoregeometry
\

AROME indique des tendances positives (+25 à >+150%) dans la vallée du Rhône et les pré-Alpes en février (**FEV**), sur l'arc méditérannéen ouest (Roussillon–Bas Languedoc) en mars (**MAR**) et est (bassin azuréen) en novembre (**NOV**). Les stations confirment ce schéma, en renforçant nettement le signal sur la vallée du Rhône en février, avec des tendances environ deux fois plus élevées qu’AROME. En juin (Annexes 2-4.3.2.2, **JUI**), AROME et les stations affichent un signal très majoritairement haussier sur l’ensemble du territoire, avec des tendances locales pouvant dépasser +150%. Cependant, la corrélation spatiale extrêmement faible entre les deux jeux de données (*r =* 0,07) indique que, si le signe moyen converge, la localisation des tendances est incohérente/bruitée. Cette discordance, combinée à la forte variabilité convective estivale et à la faible longueur des séries mensuelles (1990–2022), appelle à la prudence.

#### Corrélation des tendances relatives entre le modèle AROME et les stations Météo-France

Pour objectiver ces divergences et mesurer la robustesse des signaux, nous examinons désormais les corrélations spatiales entre AROME et les stations, par saison et mois, afin d’identifier où l’accord est structurel et où il relève surtout du bruit. \

À l’échelle quotidienne (1959‑2022), les corrélations des tendances se situent entre **0,14** (HYDRO) et **0,23** (DJF) pour les saisons et entre **0,07** (JUI) et **0,51** (DEC) pour les mois, plusieurs mois atteignant ou dépassant **0,40** (JAN, MAR, AOU, NOV) (Figure 9). Sur la période restreinte (1990‑2022), les valeurs saisonnières couvrent **0,13–0,29** et les valeurs mensuelles **0,02–0,47** (JAN **0,47** ; DEC **0,35**). À l’échelle horaire (1990‑2022), les saisons présentent des corrélations faibles comprises entre **–0,08** (MAM) et **0,05** (DJF et SON) et les corrélations suivant les mois s’étendent de **0,02** (SEP, OCT) à **0,42** (FEV).

In [ ]:
plot_r_by_season(combined_all , echelles=['quotidien', 'quotidien_reduce', 'horaire'], col_values=['z_T_p_signif'], nom_fichier="histo_z_T_p_signif")
plot_r_by_season(combined_all , echelles=['quotidien', 'quotidien_reduce', 'horaire'], col_values=['z_T_p', 'z_T_p_signif'], nom_fichier="histo_z_T_p")

![](figures/histo_z_T_p.png){width=100%}

\captionof{figure}{\small Corrélations des tendances relatives (toutes et significatives) entre AROME et les stations Météo‑France, par saison et par mois.} 
\

En restreignant aux tendances significatives par vraisemblance profilée, l’échelle quotidienne (1959‑2022) affiche des corrélations saisonnières de **0,19** (MAM) à **0,39** (DJF) et des valeurs mensuelles de **0,17** (JUI) à **0,77** (DEC), avec d’autres mois marquant une corrélation élevée (JAN **0,70** ; MAR **0,71** ; NOV **0,68** ; AOU **0,52**). Pour la période restreinte (1990‑2022), les valeurs saisonnières vont de **0,27** (MAM) à **0,44** (DJF) et les valeurs mensuelles de **0,17** (JUILL) à **0,69** (JAN), avec des valeurs supérieures à 0,60 en février, octobre et décembre. À l’échelle horaire (1990‑2022), les valeurs saisonnières couvrent **–0,20–0,13** et les valeurs mensuelles **–0,01–0,66**. La restriction aux tendances significatives augmente fortement les corrélations quotidiennes, avec des maxima mensuels élevés. Le gain est marqué en hiver et en fin d’été / automne (JAN, MAR, NOV, DEC, AOU). À l’échelle horaire, même après filtrage, l’amélioration reste partielle : quelques mois s'améliorent (FEV) mais plusieurs périodes sont marquées par des corrélations proches de zéro ou légèrement négatives (MAI, SEP).

##### Saisonnalité récurrente de la performance

L'hiver (DJF) et les mois hivernaux isolés (DEC, JAN, MAR) concentrent les plus fortes corrélations, que ce soit sur la période longue ou restreinte, et surtout après filtrage. Le printemps (MAM) et le début d’été (JUI, JUILL) affichent systématiquement les valeurs les plus basses (ou minima) dans chaque configuration. La fin d’été / automne (AOU, NOV) fournit des corrélations intermédiaires à élevées une fois les tendances significatives retenues.

##### Hiérarchie claire des échelles temporelles

L'étude journalière est systématiquement plus cohérente spatialement pour les tendances que l'étude horaire. Le filtrage de significativité transforme la distribution journalière (multiplication des mois >0,60), alors qu’il ne suffit pas à hisser l’horaire à un niveau comparable (un seul mois >0,60).

## Discussion

#### Fidélité spatiale de la climatologie simulée

Les résultats confirment qu’AROME reproduit correctement les grands régimes pluviométriques hexagonaux confirmé par la littérature [@Fumiere2020], [@caillaud2021simulation], [@hess-28-2579-2024], [@LucasPicher2024]. Il y a un excédent orographique sur les Alpes, les Pyrénées et le Massif central, un gradient atlantique‑continental marqué à l’ouest et un déficit fréquentiel sur le pourtour méditerranéen. Cette cohérence avec la réalité mesurée témoigne d’une représentation satisfaisante des forçages dynamiques (transport d’humidité par les flux d’ouest, soulèvement orographique, circulation de basse couche en Méditerranée). 

#### Variabilité saisonnière et représentation des extrêmes

La capacité d’AROME à restituer la fréquence et la quantité de précipitations se maintient tout au long de l’année, mais la performance chute pour la moyenne des maxima journaliers en été et davantage à l’échelle horaire. On retrouve le fait qu'AROME sous-estime des précipitations d'intensité élévées (> 40 mm/h [@Caillaud2021], [@poncet2024convection]) La convection estivale reste partiellement sous‑résolue malgré la résolution spatiale de 2,5 km. Dans cette étude, on a pu montrer (résultats non affichés) que la corrélation augmente lorsque la fenêtre temporelle s'agrandit à 6 ou 9h. Le modèle pourrait reproduire la cellule orageuse en démarrant trop tard ou trop tôt, en étalant l’intensité sur plusieurs mailles (non évalué par manque de stations) ou en sous‑estimant les précipitations maximales. Le pas de 2–3 km est une étape majeure pour représenter la convection sans paramétrage, mais il reste trop grossier pour certaines applications sensibles aux maxima intenses [@Prein2015Review]. Il aurait été intéressant d'introduire les données COMEPHORE (1 km, 15 min) de Météo-France dans cette étude mais les réanalyses radar ne débutent qu'en 1997 avec une qualité médiocre sur les Alpes avant 2007 [@Fumiere2020].

#### Tendances des extrêmes de précipitations : cohérences, divergences et limites

##### Données journalière

La tendance positive et significative sont cohérentes avec l'étude à l'échelle terrestre de l'@IPCC2021 montrant une hausse du niveau de retour 10 ans de +6,7% issu des pluies journalières pour 71% de la surface. Les hot-spots sur le Mercantour (+20 à +30 %) et, côté stations, sur la vallée du Rhône (+5 à < +30 %) sont cohérents avec les travaux montrant une intensification des extrêmes dans le Sud‑Est de la France et sur les Alpes du Sud de @blanchet2021explaining. En automne une augmentation du niveau de retour 20 ans pourrait atteindre l’ordre de grandeur de sa valeur moyenne dans le sud‑est alpin. Les tendances haussières marquées des stations (> +35%) sur la moitié Nord de la France résonnent avec les projections françaises qui anticipent des augmentations plus fortes des extrêmes journaliers dans le Nord sous réchauffement (+20% pour +4°C) montré par @soubeyroux:hal-04991790. Le biais moyen négatif (−2,4 % à −4,9 %) signifie que le modèle aplatit systématiquement les tendances par rapport aux observations, à relier à la représentation des extrêmes par AROME.

##### Données horaires

Les signaux sont beaucoup moins robuste au pas de temps horaire qu'au pas de temps journalier. Ils montrent des tendances très hétérogènes, souvent peu ou pas significatives, et parfois incohérentes spatialement entre AROME et les stations (en juin, *r =* 0,07). Cela rejoint le constat de @IPCC2021 concernant la faible confiance dans une hausse globale des extrêmes sous‑journaliers, faute de séries longues, denses et homogènes. Les hausses marquées en février (vallée du Rhône, pré‑Alpes), mars (arc méditerranéen ouest) et novembre (arc méditerranéen est) sont compatibles avec les régimes dynamiques et thermo‑marins propices aux épisodes méditerranéens/transitionnels (fin d’hiver, intersaison automnale). Le signal doublé par les stations en février sur la vallée du Rhône souligne le lissages et biais d’amplitude d’AROME sur les extrêmes horaires. Toutefois, la période restreinte (1990–2022) offre peu de puissance statistique, et la variabilité interannuelle convective peut cacher ou mimer une tendance. En juin, la hausse généralisée (jusqu’à +150%), mais sans accord spatial entre AROME et stations montre la variabilité convective locale (orages très ponctuels) et les erreurs de représentativité point *vs.* maille deviennent dominantes. C'est un exemple des limites de l’inférence mensuelle sur les extrêmes convectifs. \

DeS études régionales mettent en évidence des sensibilités infra‑journalières de +7 à +13%/°C, parfois supérieures à Clausius‑Clapeyron, notamment pour des orages convectifs brefs [@molnar2015relation]. L'étude n'identifie pas de signal clair avec le modèle AROME qui sous‑estime fortement les pics horaires, alors que les stations captent des pointes locales. C'est ce à quoi on s'attendait au regard de la part des tendances des extrêmes liée à Clausius-Clapeyron qui devait théoriquement être deux fois plus faible que les tendances observées.

#### Robustesse statistique des signaux

Une fenêtre longue (1959–2022) réduit les incertitudes d’estimation et échantillonne plusieurs régimes climatiques (phase de global dimming dominée par les aérosols jusqu’aux années 1980, puis accélération du réchauffement à partir de la fin des années 1980–1990). Ce mélange de régimes atténue mécaniquement la pente moyenne : une partie du signal récent est noyée dans la variabilité multidécennale. À l’inverse, une fenêtre courte (1990–2022) isole le régime dominé par le réchauffement rapide, la diminution des aérosols sulfatés en Europe et l’accroissement quasi linéaire du contenu en vapeur d’eau (+7%/°C), ce qui fait ressortir des tendances positives plus nettes. Les deux diagnostics sont donc complémentaires et doivent être interprétés séparément pour ne pas confondre bruit multidécennal et forçage anthropique de long terme. \

En première approximation, les distributions mensuelles des tendances relatives du niveau de retour 10 ans sont centrées sur 0% sur 1959–2022, tandis que 1990–2022 montre un léger basculement vers des accroissements, surtout au printemps et en début d’hiver, en phase avec la littérature sur l’amplification récente des extrêmes journaliers. Même si des points de rupture ont été autorisés lorsque le LRT le justifiait, la transition progressive (environ 10 ans) entre l’ère du dimming et le régime chaud post‑1985 reste en partie absorbée par les modèles longue fenêtre, ce qui réduit la pente moyenne par rapport aux modèles courte fenêtre qui se calent sur la portion quasi linéaire récente. Ceci éclaire, par exemple, la tendance nationale moyenne observée de −1,6% sur 1959–2022 contre +3,5% sur 1990–2022. \

Ces différences sont également attendues d’un point de vue statistique : @DeGaetano2018 montre que, pour des tendances > 0,5%/an sur le paramètre de localisation $\mu$, réduire la fenêtre de 60 à 30 ans peut modifier la pente estimée des niveaux de retour décennaux de 10 à 20 %. Autrement dit, la longueur de la fenêtre et la (non)‑stationnarité du climat pèsent fortement sur l’estimation des tendances, même lorsque des points de rupture détectés par LRT sont intégrés.

#### Signal hivernal, bruit horaire : le filtrage renforce la cohérence mais AROME peine sur les extrêmes fins

En ne gardant que les tendances significatives on élimine nombre de sites où le signal est dominé par le bruit climatique, améliorant la cohérence spatiale des tendances et donc les diagnostics régionaux. À l’échelle journalière, les valeurs mensuelles des corrélations AROME–stations oscillent entre 0,40 et 0,77 après filtrage par significativité, contre moins de 0,20 à l’échelle horaire. Ce saut serait à relier à la difficulté d'AROME à expliquer les maxima convectifs fins au pas de temps horaire. \

Le découpage saisonnier met en lumière une hiérarchie marquée par l’hiver (DJF) qui concentre systématiquement les corrélations les plus fortes et des hausses modérées à marquées des niveaux de retour. En hiver, les précipitations en Europe occidentale viennent des perturbations atlantiques véhiculées par les flux d’ouest. Leur intensité dépend directement de la quantité de vapeur d’eau dans ces masses d’air, qui augmente avec la température selon la loi de Clausius–Clapeyron. C’est pourquoi des hivers plus doux peuvent parfois être aussi plus pluvieux, même avec une dynamique atmosphérique identique. Le printemps et le début d’été (MAM–JUI) enregistrent au contraire les minima de corrélation systématiques. Cela révèle la difficulté persistante, même à résolution 2,5 km, à représenter la convection faiblement organisée typique de cette saison. Le réchauffement diurne déclenche surtout des orages isolés d’origine locale. L’influence des perturbations atlantiques diminue, car le jet‑stream et ses systèmes frontaux se déplacent vers le nord. Le cisaillement vertical s’affaiblit, empêchant l’organisation des orages en structures durables. Il en résulte des précipitations brèves, très localisées et peu structurées que le couple simulation–observations capture encore mal. En août et septembre, la convergence relative des distributions (médianes proches et corrélations < 0,20) traduit une variabilité élevée et une contribution partagée entre épisodes orageux continentaux et systèmes méditerranéens précoces. La mixité des mécanismes pluvieux réduit la cohérence entre sites et la capacité d’AROME à reproduire la saisonnalité des tendances.

#### Signaux régionaux : focus sur la vallée du Rhône

La vallée du Rhône semble être un « hot‑spot » des tendances positives du niveau de retour, cohérent avec la recrudescence d’événements cévenols à l’automne et de perturbations orographiques renforcées en flux de sud [@Fresnay2012]. AROME sous‑diagnostique ces hausses mais en reproduit la localisation, suggérant que la dynamique (canalisation méridienne et levée orographique) est correctement simulée, tandis que l’intensité convective demeure sous‑résolue. @Ribes2019 met en évidence que cette zone cumule le plus fort renforcement observé sur l’ensemble du Midi méditerranéen avec un gain d'intensité moyen de +22% des précipitations extrêmes journalières entre 1961 et 2015. Ils mettent en évidence sur la moitié sud‑est, incluant Gard, Ardèche et Drôme, un doublement de la fréquence des évènements >200 mm en 24h depuis 1985 avec la plupart associés à des pics horaires >50 mm. @blanchet2021explaining montrent que, depuis les années 1980, l’influence méditerranéenne automnale s’est nettement intensifiée et avancée dans la saison, avec les hausses de niveaux de retour les plus fortes centrées sur le sillon Rhône‑Alpes et les Cévennes. Ce « hot‑spot » est flagrant à l'échelle nationale pour les données horaires en février. Depuis le début des années 1990, la température moyenne des mois d'hiver en France a gagné +0,8°C (différence entre les normales 1961‑1990 et 1991‑2020 de Météo‑France) — soit un pouvoir de rétention d’humidité supplémentaire d’environ +6% selon la relation de Clausius‑Clapeyron. Trois leviers pourraient se cumuler en février : 1) la pluie remplace la neige, concentrant la lame d’eau [@ZAQOUT2024131439] ; 2) il existe des pentes de 12%/°C pour les extrêmes horaires lorsque T = 0–8°C — soit presque le double des 7% classiques [@Drobinski2016] ; et 3) des flux de sud plus humides injectent davantage de vapeur dans un couloir orographique très efficace [@LorentePlazas2020].


## Conclusion

L’évaluation conduite sur les précipitations en France montre que le modèle AROME (2,5 km, forcé par ERA5) restitue correctement les grands régimes pluviométriques observés. La climatologie simulée reproduit les contrastes majeurs du territoire : excédent orographique sur les reliefs, gradient atlantique–continental à l’ouest et déficit fréquentiel sur le pourtour méditerranéen. Cette cohérence confirme que les principaux forçages dynamiques (flux d’ouest, soulèvements orographiques, circulations méditerranéennes de basse couche) sont bien représentés. Cependant, la comparaison avec les observations révèle des **biais locaux notables**. Le modèle surestime la fréquence et les cumuls dans certains reliefs (Massif central, Pyrénées, Chamonix), et sous-estime systématiquement les extrêmes dans les Alpes du Nord, l’Alsace, la côte basque et la vallée du Rhône. Aux échelles extrêmes, les écarts atteignent −5 à −20 mm/j pour les maxima journaliers et −5 à −10 mm/h pour les maxima horaires, avec une sous-représentation marquée des événements convectifs estivaux (biais saisonnier de −3,7 mm/h en été contre −0,02 mm/h en hiver). Malgré ces écarts en valeur absolue, la corrélation spatiale reste élevée (r = 0,9 au pas journalier et 0,7–0,8 au pas horaire), montrant que la structure spatiale des précipitations est globalement captée, mais que leur intensité est aplatie. \

L’analyse des **tendances** apporte un diagnostic contrasté. À l’échelle journalière (1959–2022), les tendances positives observées dans le sud-est (Mercantour, vallée du Rhône) ou sur la moitié nord concordent avec les signaux déjà documentés, mais AROME sous-estime leur amplitude de 2 à 5 % en moyenne. À l’échelle horaire (1990–2022), les signaux apparaissent plus hétérogènes, souvent non significatifs et faiblement corrélés aux observations. Ils montrent que si des **indices locaux d’intensification existent** (vallée du Rhône en hiver, arc méditerranéen au printemps et en automne), ils restent noyés dans le bruit de la variabilité convective et ne permettent pas de conclure à une tendance généralisée. Ce constat rejoint la littérature internationale qui souligne la difficulté d’inférer des tendances robustes sur les extrêmes infra-journaliers, en raison de la longueur limitée des séries et de l’hétérogénéité des processus convectifs. En définitive, l’étude confirme que les modèles à convection explicite comme AROME constituent un progrès net pour la représentation spatiale des extrêmes et leur évaluation sur de longues périodes, mais qu’ils peinent encore à reproduire l’intensité et l’évolution des précipitations horaires extrêmes.
 
## Remerciements {.unnumbered}

Je tiens à remercier Juliette Blanchet et Antoine Blanc pour l'encadrement rigoureux, stimulant et bienveillant tout au long de ce stage. Leurs conseils avisés, leur disponibilité constante et leurs nombreuses remarques constructives m'ont permis d’approfondir considérablement mes compétences scientifiques et méthodologiques.


## References {.unnumbered}

:::{#refs}

:::



## Annexes 1 : formules mathématiques {.unnumbered}

### A.1.1. Obtention de (1)  {.unnumbered}


Soit la fonction de vraisemblance ${\displaystyle {\mathcal {L}}(\theta ;x)} : {\displaystyle \theta \mapsto f(x;\theta )}$. Alors : ${\displaystyle \log {\mathcal {L}}(\theta ;x_{1},x_{2},\dots ,x_{n})=\sum _{i=1}^{n}\log {\mathcal {L}}(\theta ;x_{i})}$.

Pour $1 + \xi \frac{x - \mu}{\sigma} > 0$, avec $\sigma > 0$ :

$$
\begin{aligned}
\log \mathcal{L}(\theta)
&= \sum_{i=1}^n \left[
  -\log \sigma
  - \frac{1 + \xi}{\xi} \log\left(1 + \xi \frac{x_i - \mu}{\sigma} \right)
  - \left(1 + \xi \frac{x_i - \mu}{\sigma} \right)^{-\frac{1}{\xi}}
\right] \\
\log \mathcal{L}(\theta)
&= -n \log \sigma
- \left(1 + \frac{1}{\xi}\right) \sum_{i=1}^n \log\left(1 + \xi \frac{x_i - \mu}{\sigma} \right)
- \sum_{i=1}^n \left(1 + \xi \frac{x_i - \mu}{\sigma} \right)^{-\frac{1}{\xi}}
\end{aligned}
$$

La log-vraisemblance $\ell(\theta) = \log \mathcal{L}(\theta)$ s’écrit alors :

$$
\ell(\theta)=
-\sum_{i=1}^n\Bigl[
\log\sigma
+\Bigl(1+\tfrac1{\xi}\Bigr)\log (1+\xi\;\frac{x_i-\mu}{\sigma})
+(1+\xi\;\frac{x_i-\mu}{\sigma})^{-\frac{1}{\xi}}
\Bigr]
\tag{1}
$$

### A.1.2. Obtention des paramètres  {.unnumbered}

$\mu_1(z_{T,1})$ et $\sigma_1(z_{T,1})$


En développant les paramètres soumis à un effet temporel, on a :

$$
\begin{aligned}
\mu_0 + \mu_1 t &= z_{T,0} + z_{T,1} t - \dfrac{\sigma_0 + \sigma_1 t}{\xi_0} \left[ \left( -\log\left(1 - \frac{1}{T} \right) \right)^{-\xi_0} - 1 \right]\\
\mu_0+\mu_1\,t &= \Bigl[\,z_{T,0}
-\dfrac{\sigma_0}{\xi_0}\Bigl(\bigl[-\log(1-\tfrac1T)\bigr]^{-\xi_0}-1\Bigr)
\Bigr]\;+\;\Bigl[\,z_{T,1}-\dfrac{\sigma_1}{\xi_0}\Bigl(\bigl[-\log(1-\tfrac1T)\bigr]^{-\xi_0}-1\Bigr)\Bigr]\,t\\
\end{aligned}
$$

c’est-à-dire, terme à terme :

$$
\begin{aligned}
\mu_0 &\;=\; z_{T,0}
-\dfrac{\sigma_0}{\xi_0}\Bigl(\bigl[-\log(1-\tfrac1T)\bigr]^{-\xi_0}-1\Bigr),\\[0.8em]
\mu_1 &\;=\; z_{T,1}
-\dfrac{\sigma_1}{\xi_0}\Bigl(\bigl[-\log(1-\tfrac1T)\bigr]^{-\xi_0}-1\Bigr).
\end{aligned}
$$

## Annexes 2 : cartographie {.unnumbered}

Afin d’alléger le corps du document, l’ensemble des cartes produites dans le cadre de cette étude n’est pas reproduit ici. Elles sont mises à disposition sous forme de figures rasterisées et vectorielles dans un répertoire en ligne accessible à l’adresse suivante :   \

https://ncsdecoopman.github.io/ExtremePrecipit/rapport/annexes.pdf   \

Ces annexes complémentaires permettent de visualiser l’ensemble des diagnostics spatiaux (détaillés par saison, pas de temps et indicateur) avec une possibilité de zoom pour examiner les motifs régionaux fins.